![title](Pictures/Title3.png)
# Comfort Zone Paradigm Analysis

In [65]:
import numpy as np
import pandas as pd
import csv
import os, shutil
import glob
import easygui
import matplotlib.pyplot as plt
from IPython.display import Image
import math
from math import sqrt
import matplotlib.mlab as mlab
import statistics as st
from matplotlib.patches import Ellipse
import matplotlib.ticker as ticker
import seaborn as sns
from math import pi
from scipy import stats, integrate
sns.set()
import warnings
warnings.filterwarnings('ignore')

In [41]:
ans = easygui.boolbox(msg='Load a collection?', title='Collection', choices=('[Y]es', '[N]o'));
if ans == True:
    listcols=[];
    path=('tag_folder/collections'); #place where new files are
    #path=('Desktop/Comfort_zone_paradigm/tag_folder/collections');
    allFiles = glob.glob(path + '/*.txt');

    for file_ in allFiles:
            base = os.path.basename(file_);
            base1 = os.path.splitext(base)[0];
            listcols.append(base1);

    choice = easygui.choicebox('Which collection would you like to load?', 'Collection', listcols);
    dates = pd.read_csv('tag_folder/collections/'+str(choice)+'.txt', index_col = False)
    #dates = pd.read_csv('Desktop/comfort_zone_paradigm/tag_folder/collections/'+str(choice)+'.txt', index_col = False)
    dates = dates.date
    noe=len(dates)
    
elif ans ==False:
    listq0 =['Number of experiments']
    no = easygui.multenterbox("Enter number of experiments", "Number of experiments", listq0)
    noe = int(no[0])
    e=1
    exp_list=[]
    for i in range(noe):
        exp= 'Experiment '+str(e)
        exp_list.append(exp)
        e=e+1

    dates = []
    dates = easygui.multenterbox('Enter experiments to be analysed', 'Experiment', exp_list)
    
    ans2 = easygui.boolbox(msg='Save as collection?', title='Collection', choices=('[Y]es', '[N]o'), image=None, default_choice='Yes', cancel_choice='No');
    if ans2 == True:
        listq1 =['Collection name']
        cn=[]
        cn = easygui.multenterbox("Enter name of collection", "Name collection", listq1)
        cn=str(cn[0])
        cntable = pd.DataFrame({'date':dates})
        #cntable.to_csv('Desktop/Comfort_zone_paradigm/tag_folder/collections/'+str(cn)+'.txt')
        cntable.to_csv('tag_folder/collections/'+str(cn)+'.txt')
    elif ans2 == False:
        pass
    
answer=easygui.ynbox('Normal experiment details?', 'Experiment details');
listq =['Time the divider was pulled','Total length of the trial','Time the trial started']
experiment_details = []
if answer == False:
    experiment_details = easygui.multenterbox('Enter the experiment details', 'Experiment details', listq)
    Time_divider_pulled = int(experiment_details[0])
    length_of_trial = int(experiment_details[1])
    time_start = int(experiment_details[2])
else:
    Time_divider_pulled = 60
    length_of_trial = 210
    time_start = 0
    
T1,T2,T0,TI = Time_divider_pulled*30,length_of_trial*30,time_start*30,length_of_trial - Time_divider_pulled

header = ['Trial_time','Recording_time','X_center','Y_center','Area','Areachange','Elongation','Distance_moved','Heading_to_point','Mobility_continuous','In_zone(Blue/Center-point)','In_zone(White/Center-point)','In_zone(Blue_1/Center-point)','In_zone(Social/Center-point)','In_zone(Blue_2/Center-point)','In_zone(Upper/Center-point)','In_zone(Lower/Center-point)','In_zone(Transition_zone/Center-point)','Result_1','Result_2']
listall=[]
allFiles=[]

listallconditions=[]
listalltreatment=[]
listallsubject=[]
listallstrain=[]

for e in range(noe):
    list_=[]
    path=('Temp_bin/' + dates[e]) #place where new files are
    #path=('Desktop/Comfort_zone_paradigm/Temp_bin/' + dates[e]) #place where new files are
    allFiles = glob.glob(path + '/*.txt')
    if len(allFiles)>0:
        frame = pd.DataFrame()
        hi=1
        for file_ in allFiles:
            hi=hi+1
            #print(hi)
            df = pd.read_csv(file_, names=header,index_col=False, encoding ='utf-16',  skiprows=36)
            for dfs in df:
                try: 
                    df=df[df.Area != '-']
                except: 
                    pass
                base = os.path.basename(file_)
                base1 = os.path.splitext(base)[0]
                df['Trial'] =  base1
            list_.append(df)
    else:
        easygui.msgbox('Reload data files into temp_bin folder and try again')
    h=1
    listdf=[]        
    for df1 in list_:
        
        h=h+1
        df1['blue']= df1['In_zone(Blue/Center-point)'] ## renaming zone columns to simpler names
        df1['white']= df1['In_zone(White/Center-point)']
        df1['social']= df1['In_zone(Social/Center-point)']
        df1['transition']= df1['In_zone(Transition_zone/Center-point)']
        df1 = df1.filter(['Trial_time','Recording_time','X_center','Y_center','blue','white','social','transition'], axis=1) #filters columns
        df1 = df1.reset_index(drop=True) #resets index column to match new row count
        df1 = df1.astype(float) #float it
        listdf.append(df1)
        

    one,two,listm,length,trial_list,trial = 0,1,[],int((len(listdf))/2),[],100
    for i in range(length):
        df1=listdf[one]
        df2=listdf[two]
        dfm=pd.merge(df1,df2,on=['Trial_time','Trial_time'],how="outer")
        
        dfm['Trial_time2'] = dfm.Trial_time
        
        dfm = dfm.set_index('Trial_time2') ##########################
        dfm = dfm.sort_index() ######################################
        
        listm.append(dfm)
        
        one = one+2
        two = two+2
        trial=trial+1
        trial_list.append(trial)
        
    
    listm2=[]
    for df1 in listm:
        listn=[]
        n=0
        for i in range(0, len(df1.transition_y)):
            listn.append(n)
            n = n+1
        df1['Column 1'] = listn
        
        df1 = df1.set_index('Column 1') ##########################

        listm2.append(df1)
        
     
    

    listdf=[]
    for df1 in listm2:
        df1 = df1.astype(float)
        listx,listx2=[],[]
        for i in range(0, len(df1.Trial_time)):
            x = ((df1.X_center_x[i] + 26)*1.188)-1.5
            listx.append(x)
            x2 = 24.2 - df1.X_center_y[i]
            listx2.append(x2)
        df1['X_center_x2'] = listx
        df1['X_center_y2'] = listx2 
        listdf.append(df1)

    for df1 in listdf:
        listx=[]
        for i in range(0, len(df1.Trial_time)):
            if df1.blue_y[i] == 1:
                x = df1.X_center_y2[i]
            else:
                x = df1.X_center_x2[i]
            listx.append(x)
        df1['x']=listx
        
    listcondition=[]
    listtreatment=[]
    listsubject=[]
    liststrain=[]
    
    
    #if os.path.exists('Desktop/comfort_zone_paradigm/tag_folder/'+str(dates[e])+'.csv'):
    if os.path.exists('tag_folder/'+str(dates[e])+'.csv'):
        #tag = pd.read_csv('Desktop/comfort_zone_paradigm/tag_folder/'+str(dates[e])+'.csv', index_col = False)
        tag = pd.read_csv('tag_folder/'+str(dates[e])+'.csv', index_col = False)
        condition = []
        condition = tag.Condition
        treatment = []
        treatment = tag.Treatment
        subject = []
        subject = tag.Subject
        strain = []
        strain = tag.Strain
        
        listcondition.append(condition)
        listtreatment.append(treatment)
        listsubject.append(subject)
        liststrain.append(strain)
        
    else:
        condition = []
        condition = easygui.multenterbox('Enter condition of each trial', 'Condition', trial_list)
        treatment = []
        treatment = easygui.multenterbox('Enter treatment of each trial', 'Treatment', trial_list)
        subject = []
        subject = easygui.multenterbox('Enter subject of each trial', 'Subject', trial_list)
        strain = []
        strain = easygui.multenterbox('Enter Strain of each trial', 'Strain', trial_list)    

        tags = pd.DataFrame({'Trial':trial_list,'Condition':condition,'Treatment':treatment,'Subject':subject,'Strain':strain}) 
        tags['Date']=dates[0]
        #tags.to_csv('Desktop/Comfort_zone_paradigm/tag_folder/'+str(dates[e])+'.csv')
        tags.to_csv('tag_folder/'+str(dates[e])+'.csv')
    
    listdf1,q=[],0
    for df1 in listdf:  
        df1 = df1.reset_index()
        df1['date']= dates[e]
        df1['trial']=trial_list[q]
        df1['condition']=condition[q]
        df1['treatment']=treatment[q]
        df1['subject']=subject[q]
        df1['strain']=strain[q]
        df1['X_center'] = df1.x
        df1['X_center_side'] = df1.X_center_y
        df1['Y_center'] = df1.Y_center_y
        df1['Z_center'] = df1.Y_center_x
        df1['Recording_time'] = df1.Recording_time_y
        df1['blue'] = df1.blue_y
        df1['white'] = df1.white_y
        df1['transition'] = df1.transition_y
        df1['social'] = df1.social_y
        df1 = df1.filter(['date','trial','condition','treatment','subject','strain','Recording_time','X_center','X_center_side','Y_center','Z_center','blue','white','social','transition','distance_moved'], axis=1)
        
        
        listallconditions.append(condition[q])
        listalltreatment.append(treatment[q])
        listallsubject.append(subject[q])
        listallstrain.append(strain[q])
        
        
        listdf1.append(df1)
        listall.append(df1)
        q=q+1
        
dfc = pd.concat(listall)
dfc.to_csv('data/all.csv');
#dfc.to_csv('Desktop/Comfort_zone_paradigm/data/all.csv');

In [43]:
listdfleave=[]
listdf=listall
list_df=[]
listspre2,lists2,listb2,listb2a,lists,listb,listw,listtr,listsa,listba,listwa,listtra,listsocialtag,listbluetag, listsha,listshoalraw,listcorna,listcornaraw,listmida = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
listtriage1=[]
listinteractionspider=[]
c=0
for df1 in listdf:
    
    df1=df1.fillna(0)
    listpresoc2,listsoc2,listbluepre,listwhitepre,listsocialpre,listtransitionpre,listbluepost,listwhitepost,listsocialpost,listtransitionpost = [],[],[],[],[],[],[],[],[],[]
    for i in range(0, len(df1.Recording_time)):
        if df1.Recording_time[i] < Time_divider_pulled:
            listbluepre.append(df1.blue[i])
            listwhitepre.append(df1.white[i])
            listsocialpre.append(df1.social[i])
            listtransitionpre.append(df1.transition[i])
        else:
            listbluepost.append(df1.blue[i])
            listwhitepost.append(df1.white[i])
            listsocialpost.append(df1.social[i])
            listtransitionpost.append(df1.transition[i])
    
    lengthpre = 100/(len(listbluepre))
    lengthpost = 100/(len(listbluepost))
    
    bluepre = sum(listbluepre)*lengthpre
    whitepre = sum(listwhitepre)*lengthpre
    socialpre = sum(listsocialpre)*lengthpre
    transitionpre = sum(listtransitionpre)*lengthpre
    blue2pre = bluepre + transitionpre
    
    bluepost = sum(listbluepost)*lengthpost
    whitepost = sum(listwhitepost)*lengthpost
    socialpost = sum(listsocialpost)*lengthpost
    transitionpost = sum(listtransitionpost)*lengthpost
    blue2post = bluepost + transitionpost
    
    
    if listallconditions[c] == 'Basal':
        if bluepre >= 60:
            triage1 = 'stay'
        else:
            triage1 = 'leave'
    else:
        triage1 = 'test'
    
    listtriage1.append(triage1)
    c=c+1
    
    df1['triage1'] = triage1
    
    listshoal=[]
    for i in range(0, len(df1.Recording_time)):
        if df1.Recording_time[i] > Time_divider_pulled:
            if df1.X_center[i] > 20.6 and df1.Z_center[i] > -1.9 and df1.Z_center[i] < 7.2 and df1.social[i] == 1: #
            #if df1.X_center[i] > 22 and df1.Z_center[i] > -2.6 and df1.Z_center[i] < 7.4 and df1.social[i] == 1: #
                shoal = 1
            else:
                if df1.social[i] == 1 and df1.X_center[i] == 0:
                    shoal = 1
                else:
                    shoal = 0
        else:
            shoal=0
        listshoal.append(shoal)

    shoalraw=(sum(listshoal)*lengthpost)
    
    df1['shoaling_zone'] = listshoal
    

    
    listl=[]
    
    for i in range(0, len(df1.Recording_time)):
        if df1.Recording_time[i] > Time_divider_pulled:
            l = 1
        else:
            l=0
        listl.append(l)
        
    trial_length = (sum(listl)/30)
    
    
    
    listinteraction_zone=[]
    for i in range(0, len(df1.Recording_time)):
        if df1.Recording_time[i] > Time_divider_pulled:
            #if df1.X_center[i] > 16 and df1.Z_center[i] > 7.5:
            if df1.X_center[i] > 14.5 and df1.Z_center[i] > 7.2:
                interaction_zone = 1
            else:
                #if df1.X_center[i] > 16 and df1.Z_center[i] < -2.4:
                if df1.X_center[i] > 14.5 and df1.Z_center[i] < -1.9:
                    interaction_zone = 1
                else:
                    #if df1.X_center[i] > 22.5:
                    if df1.X_center[i] > 20.6:
                        interaction_zone = 1
                    else:
                        interaction_zone = 0
        else:
            interaction_zone = 0
        listinteraction_zone.append(interaction_zone)
    
    percent_interaction_zone = sum(listinteraction_zone)*(100/(len(listinteraction_zone)))
    #print(percent_interaction_zone)
    
    
    interactionspider=((sum(listinteraction_zone)/30)*(100/trial_length))
    listinteractionspider.append(interactionspider)
    #print(len(listinteraction_zone)/30)
    #print(sum(listinteraction_zone)/30)

    df1['interaction_zone'] = listinteraction_zone
    
    
    
    listmid=[]
    listside=[]
    for i in range(0, len(df1.Recording_time)):
        if df1.Recording_time[i] > Time_divider_pulled:
            if df1.social[i] == 1:
                if df1.X_center[i] == 0:
                    side = 0
                    middle =0
                else:
                    if df1.Z_center[i] < -2.6 or df1.Z_center[i] > 7.4: #
                    #if df1.Z_center[i] < -1.9 or df1.Z_center[i] > 7.2: 
                        side = 1
                        middle=0
                    else:
                        if df1.X_center[i] < 22:
                        #if df1.X_center[i] < 10.6:
                            side = 0  
                            middle=1
                        else:
                            middle=0
                            side=0
            else:
                side = 0
                middle=0
        else:
            side = 0
            middle=0   
            
        listside.append(side)
        listmid.append(middle)
    
    sideraw=(sum(listside)*lengthpost)
    soc=(sum(listsocialpost)/30)
    shoalpost=((100/soc)*((sum(listshoal)/30)))
    midpost=((100/soc)*((sum(listmid)/30)))
    sidepost=((100/soc)*((sum(listside)/30)))

    listb.append(bluepre)
    listw.append(whitepre)
    lists.append(socialpre)
    listtr.append(transitionpre)
    listb2.append(blue2pre)
    
    listba.append(bluepost)
    listwa.append(whitepost)
    listsa.append(socialpost)
    listtra.append(transitionpost)
    listb2a.append(blue2post)
    listsha.append(shoalpost)
    listshoalraw.append(shoalraw)
    listcorna.append(sidepost)
    listcornaraw.append(sideraw)
    listmida.append(midpost)
    
    df1['middle_zone'] = listmid
    df1['side_zone'] = listside 
    df1['shoaling_zone'] = listshoal 
    
    list_df.append(df1)
    

##direction
# Calculate new true distance column 
for df1 in list_df:
    
    distance_moved=[] 
    for i in range(1,len(df1.X_center)):
        a = df1.X_center[i]- df1.X_center[i-1] #a = current x coordinate - previous x coordinate
        b = df1.Z_center[i]- df1.Z_center[i-1] #b = current y coordinate - previous y coordinate
        truedistance = sqrt(a**2 + b**2) # hypotenuse calculation
        distance_moved.append(truedistance)
    distance_moved.insert(0,0)
    df1['Distance_moved']=distance_moved # replace Distance_moved column with new true distance"

    
list_alt_freeze=[]
list_freeze_pre=[]
for df1 in list_df:
    
    list_freezing=[] 
    freezing_pre=[]
    alt_freeze=[]
    for i in range(0,len(df1.Distance_moved)):
        if df1.Recording_time[i] > Time_divider_pulled:
            if df1.Distance_moved[i] <= 0.02:
                freezing = 1 
            else:
                freezing = 0
        elif df1.Recording_time[i] < Time_divider_pulled:
            if df1.Distance_moved[i] <= 0.02:
                freezing = 1
                freezing_pre.append(freezing)
                if df1.social[i]==1.0 or df1.white[i]==1.0:
                    alt=1
                    alt_freeze.append(alt)
                else:
                    pass
            else:
                freezing = 0
        list_freezing.append(freezing)
    df1['freezing']=list_freezing
    freeze_pre =sum(freezing_pre)/30
    list_freeze_pre.append(freeze_pre)
    alt_freeze_sum = sum(alt_freeze)/30
    list_alt_freeze.append(alt_freeze_sum)
    
list_latency=[]
for df1 in list_df:
    list_time=[]
    for i in range(0,len(df1.X_center)):
        if df1.Recording_time[i] > Time_divider_pulled:
            if df1.shoaling_zone[i] ==1:
                time = df1.Recording_time[i]
                list_time.append(time)
            else:
                pass
    try:
        if min(list_time)< 180:
            latency=min(list_time)
        else:latency=180
    except:
        latency=180
    list_latency.append(latency)
    
table = pd.DataFrame({'Condition':listallconditions,'Treatment':listalltreatment,'Subject':listallsubject,'Strain':listallstrain,'blue':listb2,'white':listw,'social':lists,'freeze':list_freeze_pre,'non_blue_freeze':list_alt_freeze})       
table.to_csv('data/zone_pre.csv')
#table.to_csv('Desktop/Comfort_zone_paradigm/data/zone_pre.csv')
table2 = pd.DataFrame({'Condition':listallconditions,'Treatment':listalltreatment,'Subject':listallsubject,'Strain':listallstrain,'blue':listb2a,'white':listwa,'social':listsa, 'shoal':listsha, 'shoal_raw':listshoalraw, 'side':listcorna,'sideraw':listcornaraw, 'middle':listmida})
#table2.to_csv('Desktop/Comfort_zone_paradigm/data/zone_post.csv')
table2.to_csv('data/zone_post.csv')
spider = pd.DataFrame({'Condition':listallconditions,'Treatment':listalltreatment,'Subject':listallsubject,'Strain':listallstrain,'blue_pre':listb2,'blue':listb2a,'social':listsa, 'shoal':listsha,'freeze':list_freeze_pre, 'Latency':list_latency, 'interaction_zone':listinteractionspider}) 

    
    
dfc = pd.concat(listdf)
#dfc.to_csv('Desktop/Comfort_zone_paradigm/data/interaction_zone_df.csv');                    
                        
dfc.to_csv('data/interaction_zone_df.csv');                    
                        

In [64]:
for i in range(0, len(df1.elongation)):
    #if df1.heading[i] < 10 and df1.heading[i] > -10:
    if df1.heading[i] > 140:
        print(df1.Recording_time[i])
        
print (min(df1.heading))
print (max(df1.heading))

1.166
2.966
3.466
4.999
5.499
5.5329999999999995
6.666
9.866
9.933
9.966000000000001
12.665999999999999
14.732999999999999
15.366
15.899000000000001
17.566
17.699
17.933
17.999000000000002
18.133
18.833
19.066
22.566
23.266
23.465999999999998
23.599
23.799
23.999000000000002
24.666
24.799
24.999000000000002
25.266
25.933000000000003
26.833000000000002
27.366
27.465999999999998
27.599
28.233
29.199
29.465999999999998
29.499000000000002
29.599
29.666
29.799
29.866
29.933000000000003
30.266
30.399
31.066
31.399
31.465999999999998
31.599
34.332
37.431999999999995
37.632
41.732
42.032
42.232
45.099
46.099
47.199
47.931999999999995
49.466
49.699
49.931999999999995
50.066
50.566
50.632
51.799
52.398999999999994
52.431999999999995
52.666000000000004
53.36600000000001
53.466
53.599
53.998999999999995
54.498999999999995
54.566
55.199
55.898999999999994
58.066
58.532
59.166000000000004
61.066
63.498999999999995
63.566
63.599
63.666000000000004
63.732
64.09899999999999
64.29899999999999
64.366
64.

In [56]:
for i in range(0, len(df1.elongation)):
    if df1.elongation[i] > 0.80:
        print(df1.Recording_time[i])
    pass

print (min(df1.elongation))
print (max(df1.elongation))
print (max(df1.elongation)-(max(df1.elongation)/2))

0.466
1.633
2.133
5.066
5.099
5.132999999999999
5.166
6.733
6.766
6.7989999999999995
6.832999999999999
8.366
8.399
8.433
8.466000000000001
8.499
8.533
8.566
8.599
8.833
8.866
8.899
8.933
8.966000000000001
8.999
9.066
10.333
10.366
13.933
13.966
13.999
14.133
14.165999999999999
14.199000000000002
14.232999999999999
14.265999999999998
14.299000000000001
14.333
14.366
14.399000000000001
14.433
14.466
14.499
14.533
14.565999999999999
14.599
14.633
14.665999999999999
14.699000000000002
14.732999999999999
14.765999999999998
14.799000000000001
14.833
14.866
14.899000000000001
14.933
14.966
14.999
15.033
15.065999999999999
15.099
15.133
15.165999999999999
15.232999999999999
15.299000000000001
15.333
15.366
15.399000000000001
15.499
15.533
15.565999999999999
15.599
15.633
15.665999999999999
15.699000000000002
15.799000000000001
15.833
15.866
15.899000000000001
16.066
16.099
16.133
16.166
16.199
16.233
16.266
25.699
26.599
26.699
26.899
27.433000000000003
27.465999999999998
56.266000000000005
56

In [6]:
for df1 in list_df:
    distance_moved=[] 
    for i in range(1,len(df1.X_center)):
        a = df1.X_center[i]- df1.X_center[i-1] #a = current x coordinate - previous x coordinate
        b = df1.Z_center[i]- df1.Z_center[i-1] #b = current y coordinate - previous y coordinate
        truedistance = sqrt(a**2 + b**2) # hypotenuse calculation
        distance_moved.append(truedistance)
    distance_moved.insert(0,0)
    df1['Distance_moved']=distance_moved # replace Distance_moved column with new true distance"

    distance_moved2=[] 
    for i in range(1,len(df1.X_center)):
        a = df1.X_center_side[i]- df1.X_center_side[i-1] #a = current x coordinate - previous x coordinate
        b = df1.Y_center[i]- df1.Y_center[i-1] #b = current y coordinate - previous y coordinate
        truedistance = sqrt(a**2 + b**2) # hypotenuse calculation
        distance_moved2.append(truedistance)
    distance_moved2.insert(0,0)
    df1['Distance_moved2']=distance_moved2 # replace Distance_moved column with new true distance"

In [7]:
listsumlocomotion=[]
listsumlocomotion1=[]
listsumlocomotion2=[]
for df1 in list_df:
    df2 = df1[df1.Recording_time > Time_divider_pulled]
    total = df2.Distance_moved.sum()
    total2 = df2.Distance_moved2.sum()
    listsumlocomotion1.append(((total)))
    listsumlocomotion2.append(((total2)))
    listsumlocomotion.append(((total+total2)))
    
    #print((total+total2)/len())

In [8]:
print(listsumlocomotion)

[1668.0008071368813, 2486.7589720585465, 2488.9433288784094, 1667.6835987482098, 2168.725319643977, 2111.3188021185824, 687.6053765457211, 1301.0909189494112, 645.777549567697, 1393.106159574194, 1579.579996612406, 1713.5596976212944, 1511.2385393920185, 1864.3609483446455, 1557.4007854391714, 1779.7923416822694, 2482.3675893181644, 1785.1192286432256, 643.1733074200947, 2142.2652168555433, 1814.4626354013235, 1971.718924782861, 36.07275063449086, 20.76003576656406, 48.85533450449098, 329.8890052305852, 1000.9563422412932, 720.4216811536689, 1129.7652985660975, 997.5357315864615, 873.1246481207488, 29.36796329326885, 789.6718266478615, 894.1711176660372, 639.2839792303254, 27.577148770443735]


# Zones calculations

In [9]:
listlistbluereturn=[]
for df1 in list_df:
    so = 0
    listbluereturn=[]
    for i in range(0,len(df1.X_center)):
        if df1.Recording_time[i] > 60 and df1.Recording_time[i] < 180:
            if df1.social[i] == 1:
                so = 1 
            else:
                if df1.blue[i] == 1 and so == 1:
                    bluereturn = df1.Recording_time[i]
                    so = 0
                    listbluereturn.append(bluereturn)
                else:
                    pass
        else:
            so = 0
            
    listlistbluereturn.append(len(listbluereturn))   
            
spider['returns2blue'] = listlistbluereturn
            
            
        

# Distance

In [10]:
listshoal,listside,listmid=[],[],[]
for df1 in list_df:
    listshd,listsd,listmd=[],[],[]
    for i in range(0,len(df1.X_center)):
        shoaldist=0
        sidedist=0
        middledist=0
        if df1.Recording_time[i] > Time_divider_pulled:
            if df1.shoaling_zone[i] ==1:
                shoaldist = df1.Distance_moved[i]
            else:
                if df1.side_zone[i] ==1:
                    sidedist = df1.Distance_moved[i]
                else:
                    if df1.middle_zone[i] ==1:
                        middledist = df1.Distance_moved[i]

                    else:
                        shoaldist=0
                        sidedist=0
                        middledist=0
        else:
            shoaldist=0
            sidedist=0
            middledist=0
        listshd.append(shoaldist)
        listsd.append(sidedist)
        listmd.append(middledist)
    shoal=sum(listshd)
    side=sum(listsd)
    mid=sum(listmd)
    listshoal.append(shoal)
    listside.append(side)
    listmid.append(mid)
        
table3 = pd.DataFrame({'Condition':listallconditions,'Treatment':listalltreatment,'Subject':listallsubject,'Strain':listallstrain,'shoaldist':listshoal,'sidedist':listside,'middledist':listmid})
#table3.to_csv('Desktop/Comfort_zone_paradigm/data/zone_dist.csv') 
table3.to_csv('data/zone_dist.csv') 

# Depth

In [11]:
listdrange, listmed, listaverage, liststd, listupQ, listlowQ =[],[],[],[],[],[]
for df1 in list_df:
    listdepth=[]
    for i in range(0, len(df1.X_center_side)):
        if df1.shoaling_zone[i]==1:
            depth = df1.Y_center[i] + 4.5
            listdepth.append(depth)
        else:
            pass
        
    try:
        drange = max(listdepth)- min(listdepth)
        drange = (100/6)*drange
    except:
        drange = 0

    try:
        median = st.median(listdepth)
        average = st.mean(listdepth)
    except:   
        median = 0
        average = 0
  
    try:   
        std = st.stdev(listdepth)
        lowerQ = np.percentile(listdepth, 25)
        upperQ = np.percentile(listdepth, 75)
    except: 
        std = 0
        lowerQ = 0
        upperQ = 0
    
    listdrange.append(drange)
    listmed.append(median)
    listaverage.append(average)
    liststd.append(std)
    listupQ.append(upperQ)
    listlowQ.append(lowerQ)
    
dfd = pd.DataFrame()
dfd['range']=listdrange
dfd['median']= listmed
dfd['average']=listaverage 
dfd['std']=liststd 
dfd['upperQ']=listupQ 
dfd['lowerQ']=listlowQ 

#dfd.to_csv('Desktop/Comfort_zone_paradigm/data/depth.csv');
dfd.to_csv('data/depth.csv');

spider['depth_range'] = listdrange
spider['depth_mean'] = listaverage

# Interaction calculations

In [12]:
for df1 in list_df: 
    listdirection=[]
    for i in range(1,len(df1.X_center)):
        xdif = df1.X_center[i]- df1.X_center[i-1]
        ydif = df1.Z_center[i]- df1.Z_center[i-1]
        
        boutdi = ((math.atan(xdif/ydif))*180/3.1415926)
        boutd = sqrt(boutdi**2)
        
        listdirection.append(boutdi)
        
    listdirection.insert(0,0)
    df1['direction']= listdirection

for df1 in list_df: 
    listturn=[]
    for i in range(0,len(df1.X_center)):
        if df1.shoaling_zone[i] == 1 and df1.direction[i] > 30:
            turn = 1
        else:
            turn=0
        listturn.append(turn)
    df1['turn']= listturn

for df1 in list_df: 
    listinteraction=[]
    listID=[]
    interaction = 0
    ID=0
    I=0
    for i in range(2,len(df1.X_center)):
        if df1.shoaling_zone[i] == 1:
            if df1.Recording_time[i] > Time_divider_pulled:
                if df1.Z_center[i-1] > df1.Z_center[i-2] and df1.Z_center[i] < df1.Z_center[i-1]:
                    interaction = 0
                    I=I+1
                    ID = I
                else:
                    if df1.Z_center[i-1] < df1.Z_center[i-2] and df1.Z_center[i] > df1.Z_center[i-1]:
                        interaction = 0
                        I=I+1
                        ID = I
                    else:
                        interaction = interaction+1
                        ID = I
            else:
                interaction=0
                ID=0
        else:
            interaction = 0
            ID=0
        listinteraction.append(interaction)
        listID.append(ID)
    listID.insert(0,0)
    listID.insert(0,0)
    listinteraction.insert(0,0)
    listinteraction.insert(0,0)
    df1['interaction'] = listinteraction
    df1['interaction_ID'] = listID

for df1 in list_df: 
    listID=[]
    ID=0
    I=0
    for i in range(0,len(df1.X_center)):
        if df1.shoaling_zone[i] == 1:
            if df1.Recording_time[i] > Time_divider_pulled:
                if df1.interaction[i]==1:
                    I=I+1
                    ID = I
                else:
                    ID=I
            else:
                ID=0
        else:
            ID=0
        listID.append(ID)  
    df1['interaction_ID'] = listID

for df1 in list_df: 
    df1['interaction_length'] = df1.groupby('interaction_ID')['interaction'].transform('max')
for df1 in list_df: 
    df1['interaction_shoal'] = df1.groupby('interaction_ID')['shoaling_zone'].transform('sum')

for df1 in list_df: 
    listshoal=[]
    for i in range(0,len(df1.X_center)):
        if df1.interaction_shoal[i] > df1.interaction_length[i]:
            shoal = 1
        else:
            shoal = 0
        listshoal.append(shoal)
    df1['shoal']=listshoal

for df1 in list_df:
    listPC=[]
    count=0
    for i in range(1,len(df1.X_center)):
        if df1.interaction[i]==1:
            if df1.interaction_length[i] > 5 and df1.shoal[i]==1:
                count = 1
                PC = count
            else:
                count = count+1
                PC=count
        else:
            if df1.interaction[i]>1:
                count = count+1
                PC=count
                
            else:
                if df1.interaction[i]==0 and df1.interaction[i-1] >0 and df1.shoaling_zone[i]==0:
                    count=1
                    PC=count
                else:
                    if df1.interaction[i]==0 and df1.interaction[i-1]==0:
                        count = count+1
                        PC=count
                    else:
                        PC=count
                        
        listPC.append(PC)
    listPC.insert(0,0)
    df1['PCcount']=listPC
                
listints, listintslen =[],[]
for df1 in list_df:
    listint=[]
    
    for i in range(1,len(df1.X_center)):
        try:
            if df1.PCcount[i+1] == 1:
                interval = df1.PCcount[i]/30
                listint.append(interval)
            else:
                if df1.PCcount[i] == 1 and df1.shoal[i] == 1 and df1.shoal[i-1] == 1:
                    interval = 0.033
                    listint.append(interval)

                else:
                    pass
        except:
            pass
    try:
        listint.pop(0)
    except:
            pass
    listints.append(listint)
    listintslen.append(len(listint))

#spider['interaction_frequency']=listintslen

dfc = pd.concat(list_df)
#dfc.to_csv('Desktop/Comfort_zone_paradigm/data/concat_'+str(dates[0])+'.csv');
dfc.to_csv('data/concat_'+str(dates[0])+'.csv');

# Interaction intervals and breaks

In [13]:
list_interval_duration_average=[]
list_interval_distance_average=[]
list_interval_duration_freq=[]
list_interval_speed_average=[]
for df1 in list_df:
    listinterval_duration=[]
    listinterval_distance=[]
    intd=0
    dist=0
    listintd=[]
    listdist=[]
    for i in range(1,len(df1.X_center)):
        if df1.shoaling_zone[i]==1 and df1.PCcount[i] ==1:
            intd=0
            dist=0
        else:
            if df1.shoaling_zone[i-1]==0 and df1.shoaling_zone[i]==1:
                intd = 0
                dist=0
            else:
                if df1.shoaling_zone[i]==1:
                    intd=intd+1
                    dist=df1.Distance_moved[i]+dist
                else:
                    if df1.shoaling_zone[i]==0:
                        intd=0
                        dist=0
                    else:
                        intd=0
                        dist=0
        listintd.append(intd)
        listdist.append(dist)
        
    listintd.insert(0,0)
    listdist.insert(0,0)
    
    df1['listintd']=listintd
    df1['listdist']=listdist
    
    for i in range(1,len(df1.X_center)):
        if df1.Recording_time[i]>60 and df1.Recording_time[i]<180:
            if df1.listintd[i-1] >0 and df1.listintd[i] == 0:
                interval_duration= (df1.listintd[i-1]/30)
                listinterval_duration.append(interval_duration)
    
    try:
        interval_duration_freq =(len(listinterval_duration))
    except:
        interval_duration_freq=0
    list_interval_duration_freq.append(interval_duration_freq)

print(list_interval_duration_freq)    
#interactions = pd.DataFrame({'int_duration':list_interval_duration_average,'int_number':list_interval_duration_freq})
    
spider['int_number']=list_interval_duration_freq


list_break_duration_average=[]
for df1 in list_df: 
    breakid = 1
    breakd = 0
    listbreakd=[]
    listbreakid=[]
    for i in range(1,len(df1.X_center)):
        if df1.shoaling_zone[i]==0 and df1.shoaling_zone[i-1]==1:
            breakd = 0
            breakid =0
        else:
            if df1.shoaling_zone[i]==0:
                if breakd == 0:
                    breakid = breakid+1
                else:
                    breakid = breakid
                breakd = breakd+1
                
            else:
                breakd = 0
                breakid =0       
        listbreakd.append(breakd)
        listbreakid.append(breakid)
    listbreakd.insert(0,0)
    listbreakid.insert(0,0)
    df1['breakd']=listbreakd
    df1['breakid']=listbreakd
for df1 in list_df: 
    breakd = 0
    listbreak_duration=[]
    for i in range(1,len(df1.X_center)):
        if df1.PCcount[i] ==1 and df1.shoaling_zone[i]==1 and df1.shoaling_zone[i-1]==1:
        
            break_duration=0.0333
            listbreak_duration.append(break_duration)
        else:
            if df1.breakd[i]==0 and df1.breakd[i-1] >1:
                break_duration=df1.breakd[i-1]/30
                listbreak_duration.append(break_duration)  
            else:
                pass
        
    try:
        break_duration_average =(sum(listbreak_duration)/len(listbreak_duration))
    except:
        break_duration_average=0
    list_break_duration_average.append(break_duration_average)   
    
#interactions['break_duration']=list_break_duration_average
#spider['break_duration']=list_break_duration_average


listdate=[]
for df1 in list_df:
    date=df1.date[1]
    listdate.append(date)
spider['date']=listdate

spider.to_csv('data/Spider.csv')
#spider.to_csv('Desktop/Comfort_zone_paradigm/data/Spider.csv')

dfc = pd.concat(list_df)
#dfc.to_csv('Desktop/Comfort_zone_paradigm/data/all_a.csv');
dfc.to_csv('data/all_a.csv');

[39, 55, 41, 67, 83, 35, 5, 21, 0, 110, 48, 66, 59, 51, 63, 51, 50, 91, 8, 18, 15, 36, 0, 0, 0, 0, 24, 25, 22, 35, 26, 0, 45, 23, 0, 0]


In [14]:

#latestagesum=[]
#for df1 in list_df: 
#    list_latestage_interaction_zone = []
#    for i in range(1,len(df1.X_center)):
#        if df1.Recording_time[i]>120 and df1.Recording_time[i]<180:
#            latestage_interaction = df1.interaction_zone[i]
#            list_latestage_interaction_zone.append(latestage_interaction)
#        else:
#            pass
#    latestagesum.append((100/60)*(sum(list_latestage_interaction_zone)/30))
#spider['latestage_interaction'] = latestagesum   
#spider.to_csv('data/Spider.csv')

In [15]:
latestagesum=[]
for df1 in list_df: 
    list_latestage_interaction_zone = []
    for i in range(1,len(df1.X_center)):
        if df1.Recording_time[i]>120 and df1.Recording_time[i]<180:
            latestage_interaction = df1.interaction_zone[i]
            list_latestage_interaction_zone.append(latestage_interaction)
        else:
            pass
    latestagesum.append((100/60)*(sum(list_latestage_interaction_zone)/30))
spider['latestage_interaction'] = latestagesum   

listdate=[]
for df1 in list_df:
    date=df1.date[1]
    listdate.append(date)
spider['date']=listdate

#spider.to_csv('Desktop/Comfort_zone_paradigm/data/Spider.csv')
spider.to_csv('data/Spider.csv')

dfc = pd.concat(list_df)
#dfc.to_csv('Desktop/Comfort_zone_paradigm/data/all_a.csv');
dfc.to_csv('data/all_a.csv');

In [16]:
for i in range(0, len(table.social)):
    if (table.social[i]+table.white[i]+table.blue[i]) < 90:
        print('Blue zone tracking error: '+str(table.Condition[i])+'  '+str(table.Subject[i])+'  '+str(table.Treatment[i])+'  '+str(table.Strain[i])+'  '+str(i))
    if (table2.social[i]+table2.white[i]+table2.blue[i]) < 90:
        print('Blue zone tracking error: '+str(table2.Condition[i])+'  '+str(table2.Subject[i])+'  '+str(table2.Treatment[i])+'  '+str(table2.Strain[i])) 

In [17]:
up = 180
down = 120

for df1 in list_df: 
    list_interaction_zone_binned = []
    for i in range(0,len(df1.X_center)):
        if df1.condition[i]=='Basal':
            if df1.strain[i]=='TU':
                if df1.Recording_time[i]>down and df1.Recording_time[i]<up:
                    interaction_zone_binned = df1.interaction_zone[i]
                    list_interaction_zone_binned.append(interaction_zone_binned)
                    
                else:
                    pass
            else:
                pass
        else:
            pass
    if len(list_interaction_zone_binned)>1:
        print(((100/(up-down))*(sum(list_interaction_zone_binned)/30)))


In [18]:
intlengthlist=[]
numberofclusterslist=[]
intdistancelist=[]
averageclusterlist=[]
lenghtofclusterslist=[]

for df1 in list_df:
    n=0
    breaklen = 0
    listbreakid,listbreaklen=[],[]
    for i in range(1,len(df1.X_center)):
        if df1.interaction[i] == 0:
            if df1.interaction[i-1] > 0:
                n=n+1
                breakid = n
                breaklen = 1
            else:
                breakid = n
                breaklen = breaklen+1
        else:
            breaklen = 0
            breakid = 0

        listbreakid.append(breakid)
        listbreaklen.append(breaklen)
        
    listbreaklen.insert(0,0)
    listbreakid.insert(0,0)
    
    df1['listbreakid'] = listbreakid
    df1['listbreaklen'] = listbreaklen
    
    
    df1['listbreakid_length'] = df1.groupby('listbreakid')['listbreaklen'].transform('max')
    
    n, cluster, listcluster = 0, 0, []
    for i in range(0,len(df1.X_center)):
        if df1.Recording_time[i] > 120 and df1.Recording_time[i] < 180: ## late stage or full social?
            if df1.interaction[i] == 0 and df1.listbreakid_length[i] >= 60:
                cluster = 0
            else:
                if df1.interaction[i] == 0 and df1.listbreakid_length[i] < 60:
                    cluster = n
                else:
                    if df1.interaction[i] == 1 and cluster == 0:
                        n=n+1
                        cluster = n
                        
                    else:
                        cluster = cluster
        else:
            cluster = cluster
        listcluster.append(cluster)
        
    df1['cluster'] = listcluster
    
    #print('new subject')
    #print('number of clusters:  '+ str(df1['cluster'].max()))
    #print(df1['cluster'].max()) ## number of clusters
    
    numberofclusterslist.append((df1['cluster'].max()))
    
    df1['int_max'] = df1.groupby('cluster')['interaction_ID'].transform('max')
    df1['int_min'] = df1.groupby('cluster')['interaction_ID'].transform('first')
    
    df1['time_max'] = df1.groupby('cluster')['Recording_time'].transform('max')
    df1['time_min'] = df1.groupby('cluster')['Recording_time'].transform('first')
    
    listtimecluster, listrangecluster,listrange, listtime,listintfrequency =[],[],[],[],[]
    for i in range(1,len(df1.X_center)):
        rangecluster = (df1.int_max[i] - df1.int_min[i])+1
        timecluster = (df1.time_max[i] - df1.time_min[i])
        if timecluster > 60:
            timecluster = 60
        else:
            timecluster=timecluster
        listrangecluster.append(rangecluster)
        listtimecluster.append(timecluster)
        if df1.cluster[i-1] == 0 and df1.cluster[i] > 0:
            listrange.append(rangecluster)
            listintfrequency.append(rangecluster/timecluster)
            if df1.Recording_time[i] > 120 and df1.Recording_time[i] < 180:
                listtime.append(timecluster)
            else:
                pass
        else:
            listintfrequency.append(rangecluster/timecluster)
            pass

        
    listrangecluster.insert(0,0)
    listintfrequency.insert(0,0)
    df1['rangecluster'] = listrangecluster
    df1['intfrequency'] = listintfrequency
    
    try:
        #print(sum(listtime)/len(listtime))  ### average cluster time?
        averageclusterlist.append((sum(listtime)/len(listtime)))
        duration = (sum(listtime)/len(listtime))
    except:
        #print(0)
        duration = 0
        averageclusterlist.append(0)
        
    #print((df1['cluster'].max())*duration) ##length of clusters combined (max 60)
    
    lenghtofclusterslist.append(((df1['cluster'].max())*duration))
    
    listintdist=[]
    for i in range(1,len(df1.X_center)):
        if df1.Recording_time[i] > 120 and df1.Recording_time[i] <= 180:
            if df1.interaction[i] == 0:
                intdist=0
            else:
                intdist = df1.Distance_moved[i] + intdist
                #print(intdist)
        else:
            intdist=0
        listintdist.append(intdist)
    listintdist.insert(0,0)
    df1['listintdist'] = listintdist
    intlength=[]
    for i in range(1,len(df1.X_center)):
        if df1.Recording_time[i] > 120 and df1.Recording_time[i] <= 180:
            if df1.listintdist[i-1] > df1.listintdist[i]:
                intlength.append(listintdist[i-1])
    
    
    
    try:
        intdistancelist.append((sum(intlength)/len(intlength)))
        #print((sum(intlength)/len(intlength))) ### interaction distance (dist per int)
    except:
        #print(0)
        intdistancelist.append(0)
        
    #print(len(intlength))
    #print(len(intlength))
    intlengthlist.append(len(intlength))
    
    
spider['intlengthlist']= intlengthlist
spider['numberofclusterslist']= numberofclusterslist
spider['intdistancelist']= intdistancelist
spider['averageclusterlist']= averageclusterlist
spider['lenghtofclusterslist']=lenghtofclusterslist


dfc = pd.concat(list_df)
#dfc.to_csv('Desktop/Comfort_zone_paradigm/data/all_test.csv');    
dfc.to_csv('data/all_test.csv');    

In [19]:
bluezonelist=[]
listotherlist=[]
loiterdepthlist=[]
loiterspeendlist=[]


for df1 in list_df:
    listdepth=[]
    listspeed=[]
    listint,listblue,listother = [],[],[]
    for i in range(0,len(df1.X_center)):
        if df1.Recording_time[i] > 120 and df1.Recording_time[i] < 180:
            if df1.interaction_zone[i] == 1:
                interaction = df1.interaction_zone[i]
                blue = 0
                other = 0
                
            else:
                if df1.blue[i] == 1:
                    blue = df1.blue[i]
                    interaction = 0
                    other = 0
                else:
                    blue = 0
                    interaction = 0
                    other = 1
                    listdepth.append((df1.Y_center[i])+4)
                    listspeed.append(df1.Distance_moved[i])
        else:
            blue = 0
            interaction = 0
            other = 0
        listint.append(interaction)
        listblue.append(blue)
        listother.append(other)
        
    try:
        loiterdepthlist.append((sum(listdepth)/len(listdepth)))
        loiterspeendlist.append(((sum(listspeed)/len(listspeed))*30))
    except:
        loiterdepthlist.append(0)
        loiterspeendlist.append(0)  
        
    #print((100/60)*(sum(listint)/30))
    bluezonelist.append((100/60)*(sum(listblue)/30))
    listotherlist.append((100/60)*(sum(listother)/30))     
    
spider['bluezonelist']= bluezonelist            
spider['listotherlist']= listotherlist                      
spider['loiterdepthlist']=loiterdepthlist
spider['loiterspeendlist'] =loiterspeendlist             

In [20]:
listdelta, listtrial, listdate =[],[],[]
listcondition =[]
liststrain =[]
listdist=[]
for df1 in list_df:
    listdepth=[]
    listspeed=[]
    listint,listblue,listother = [],[],[]
    for i in range(0,len(df1.X_center)):
        if df1.Recording_time[i] > 120 and df1.Recording_time[i] < 180:
            if df1.interaction_zone[i] == 1:
                interaction = df1.interaction_zone[i]
                blue = 0
                other = 0
                
            else:
                if df1.blue[i] == 1:
                    blue = df1.blue[i]
                    interaction = 0
                    other = 0
                else:
                    blue = 0
                    interaction = 0
                    other = 1
                    listdepth.append((df1.Y_center[i])+4)
                    listspeed.append(df1.Distance_moved[i])
        else:
            blue = 0
            interaction = 0
            other = 0
        listint.append(interaction)
        listblue.append(blue)
        listother.append(other)
    
    try:
        avspeed = (sum(listspeed)/len(listspeed))
    except:
        avspeed = 0
        
    othercolumnlist=[]
    for i in range(0,len(df1.X_center)):
        if df1.Recording_time[i] > 120 and df1.Recording_time[i] < 180:
            if df1.interaction_zone[i] == 1:
                interaction = df1.interaction_zone[i]
                blue = 0
                other = 0

                
            else:
                if df1.blue[i] == 1:
                    blue = df1.blue[i]
                    interaction = 0
                    other = 0
                else:
                    blue = 0
                    interaction = 0
                    other = 1
                    if df1.Distance_moved[i] != 0:
                        try:
                            delta = ((sqrt((df1.Distance_moved[i]-avspeed)**2)))
                            if delta < 2:
                                listdist.append(df1.Distance_moved[i])
                                listdelta.append(delta)
                                listtrial.append(df1.trial[i])
                                listdate.append(df1.date[i])
                                listcondition.append(df1.condition[i])
                                liststrain.append(df1.strain[i])
                                
                        except:
                            pass
                            other = 0
                    else:
                        other = 0
        else:
             other = 0

                    
        othercolumnlist.append(other)

    df1['loitering'] = othercolumnlist
    
hyperactivity = pd.DataFrame({'date':listdate,  'trial':listtrial,  'delta':listdelta, 'condition' :listcondition, 'strain':liststrain, 'distance':listdist})

hyperactivity['avdist'] = hyperactivity.groupby('trial')['distance'].transform('mean')

#hyperactivity .to_csv('Desktop/Comfort_zone_paradigm/data/hyperactivity.csv')                 
hyperactivity .to_csv('data/hyperactivity.csv')   

In [21]:
listdelta, listtrial, listdate =[],[],[]
listcondition =[]
liststrain =[]
listdist=[]
for df1 in list_df:
    listdepth=[]
    listspeed=[]
    listint,listblue,listother = [],[],[]
    for i in range(0,len(df1.X_center)):
        if df1.Recording_time[i] > 120 and df1.Recording_time[i] < 180:
            if df1.interaction_zone[i] == 1:
                interaction = df1.interaction_zone[i]
                blue = 0
                other = 0
                
            else:
                if df1.blue[i] == 1:
                    blue = df1.blue[i]
                    interaction = 0
                    other = 0
                else:
                    blue = 0
                    interaction = 0
                    other = 1
                    listdepth.append((df1.Y_center[i])+4)
                    listspeed.append(df1.Distance_moved[i])
        else:
            blue = 0
            interaction = 0
            other = 0
        listint.append(interaction)
        listblue.append(blue)
        listother.append(other)
    
    try:
        avspeed = (sum(listspeed)/len(listspeed))
    except:
        avspeed = 0
        
    othercolumnlist=[]
    for i in range(0,len(df1.X_center)):
        if df1.Recording_time[i] > 120 and df1.Recording_time[i] < 180:
            if df1.interaction_zone[i] == 1:
                interaction = df1.interaction_zone[i]
                blue = 0
                other = 0

                
            else:
                if df1.blue[i] == 1:
                    blue = df1.blue[i]
                    interaction = 0
                    other = 0
                else:
                    blue = 0
                    interaction = 0
                    other = 1
                    if df1.Distance_moved[i] != 0:
                        try:
                            delta = ((sqrt((df1.Distance_moved[i]-avspeed)**2)))
                            if delta < 2:
                                listdist.append(df1.Distance_moved[i])
                                listdelta.append(delta)
                                listtrial.append(df1.trial[i])
                                listdate.append(df1.date[i])
                                listcondition.append(df1.condition[i])
                                liststrain.append(df1.strain[i])
                                
                        except:
                            pass
                            other = 0
                    else:
                        other = 0
        else:
             other = 0

                    
        othercolumnlist.append(other)

    df1['loitering'] = othercolumnlist
    
hyperactivity = pd.DataFrame({'date':listdate,  'trial':listtrial,  'delta':listdelta, 'condition' :listcondition, 'strain':liststrain, 'distance':listdist})

hyperactivity['avdist'] = hyperactivity.groupby('trial')['distance'].transform('mean')

hyperactivity .to_csv('data/hyperactivity.csv')                 
#hyperactivity .to_csv('Desktop/Comfort_zone_paradigm/data/hyperactivity.csv')  

In [22]:
#spider.to_csv('Desktop/Comfort_zone_paradigm/data/NEWSpider.csv')
spider.to_csv('data/NEWSpider.csv')

In [23]:
listsharpness2, listturnfrequency, listav2, listsumangle = [],[],[],[]
k=0
for df1 in list_df:
    listdirection = []
    listdirectionchange=[]
    listdirectionchangepos=[]
    for i in range(1,len(df1.X_center)):
        if df1.Recording_time[i] > 120 and df1.Recording_time[i] < 180:

            xdif = df1.X_center[i]- df1.X_center[i-1]

            zdif = df1.Z_center[i]- df1.Z_center[i-1]

            boutd = ((math.atan(xdif/zdif))*180/3.1415926)
            #print(boutd)

            if xdif > 0:
                if zdif > 0:
                    boutd = (0-boutd)

            if xdif > 0:
                if zdif <0:
                    boutd = (0-(180 - (sqrt(boutd**2))))

            if xdif < 0:
                if zdif > 0:

                    boutd = sqrt(boutd**2)

            if xdif < 0:
                if zdif < 0:
                    boutd = (180-boutd)
                    
                
            listdirection.append(boutd)
            
        else:
            listdirection.append(0)
    
    listdirection.insert(0,0)
    df1['direction_raw'] = listdirection
    
    for i in range(1,len(df1.X_center)):
        if df1.Recording_time[i] > 120 and df1.Recording_time[i] < 180:
            direction_change = (df1.direction_raw[i]-df1.direction_raw[i-1])
        else:
            direction_change = 0
            
        if direction_change < -180:
            direction_change = 360-sqrt((direction_change)**2)

        if direction_change > 180:
            direction_change = 0-(360-direction_change)
            
        listdirectionchange.append(direction_change)
        
        listdirectionchangepos.append(sqrt((direction_change)**2))
        
        
    listdirectionchange.insert(0,0)
    listdirectionchangepos.insert(0,0)
    
    df1['direction_change'] = listdirectionchange
    
    df1['direction_changepos'] = listdirectionchangepos
    df1['direction_changepos_test'] = listdirectionchangepos

    
    listp=[]
    
    p=0
    for i in range(1,len(df1.X_center)):
        if df1.loitering[i] == 1:
            if df1.direction_changepos[i] > 5:
                if df1.direction_changepos[i-1] > 5:
                    turnphase = p
                else:
                    p=p+1
                    turnphase = p
            else:
                turnphase = 0
        else:
            turnphase = 0
                 
        listp.append(turnphase)
        
    listp.insert(0,0)
    df1['turnphase'] = listp
    
    listp=[]
    p=0
    for i in range(1,len(df1.X_center)):
        if df1.turnphase[i] > 0:
            if turnphase == 0:
                p=p+1
                turnphase = p
            else:
                turnphase = p
        else:
            if df1.turnphase[i-1]>0 and df1.turnphase[i+1]>0:
                if turnphase == 0:
                    p=p+1
                    turnphase = p
                else:
                    turnphase = p
            else:
                turnphase = 0
        listp.append(turnphase)
        
    listp.insert(0,0)
    df1['turnphase2'] = listp
            
            
    df1['turn_max'] = df1.groupby('turnphase2')['direction_changepos'].transform('max')
    df1['turn_sum'] = df1.groupby('turnphase2')['direction_changepos'].transform('sum')  
    df1['turn_length'] = df1.groupby('turnphase2')['direction_changepos'].transform('count') 
    df1['turn_distance'] = df1.groupby('turnphase2')['Distance_moved'].transform('sum') 
    
    maxim = df1.groupby('turnphase2')['direction_changepos'].max()
    sumin = df1.groupby('turnphase2')['direction_changepos'].sum()
    countin = df1.groupby('turnphase2')['direction_changepos'].count()
    distin = df1.groupby('turnphase2')['Distance_moved'].sum()
    
    other =  spider.listotherlist[k]
    
    #### sharpness of turn (angle of turn divided by distacne of turn)
    listsharpness=[]
    listav=[]
    listmaxim=[]
    listsummin=[]
    for i in range(1, len(maxim)):
        
        
        if countin[i] > 3:
            listsharpness.append(sumin[i]/distin[i])
            listav.append(sumin[i]/countin[i])
            listmaxim.append((maxim[i]))
            listsummin.append(sumin[i])
    try:
        listsharpness2.append((sum(listsharpness)/len(listsharpness)))
    except:
        listsharpness2.append(0)
    listturnfrequency.append(((len(listmaxim)))/other)  # not /60 but /time in other
    
    try:
        listsumangle.append(sum(listsummin)/len(listsummin))
    except:
        listsumangle.append(0)
    try:
        listav2.append((sum(listav)/len(listav))*30)
    except:
        listav2.append(0)
    
    print(listsummin)
    k=k+1
    
    
    ## average turn angle
    
spider['listturnfrequency'] = listturnfrequency
spider['listsharpness2'] = listsharpness2
spider['listav2'] = listav2
spider['listsumangle'] = listsumangle

#print(listturnfrequency)
#print(listsharpness2) 
#print(listav2)

dfc = pd.concat(list_df)
#dfc.to_csv('Desktop/Comfort_zone_paradigm/data/all_test.csv'); 
dfc.to_csv('data/all_test.csv'); 

[101.64244228920961, 43.524524573452425, 127.24798975815506, 168.99895938453759, 56.31283143127198, 154.52921161174328, 32.89663317367385, 140.57741591605156, 70.9998193540186]
[194.58265421643165, 75.32127362475904, 71.19270831498741, 203.4633197052891, 238.4483222242284, 367.28978100611295, 57.315643650835554, 268.93187673551483, 248.72262545000103]
[246.96214701099044, 184.02169270919643, 140.7184538902523, 52.31101707821517, 221.32881363814897, 55.322917508336786, 207.99515900537006, 302.66681467068776, 90.2150812123916, 43.20944789986595, 235.8142504630202, 192.2412870243874, 30.018585790135063]
[49.65844522273699, 235.50684883712648, 24.140992264513955]
[194.05179931556194, 110.33614235366868, 202.54530347591606, 234.85104130743775, 196.65176349444613, 195.90861215892545]
[56.367251585571196, 126.67666751295053, 112.68903259609738, 65.844258532043, 159.88597388048396, 78.97737432253777, 86.82371278645194]
[]
[104.62462056062682, 117.72690124005236, 42.09480845253799, 76.579421052

In [24]:
#spider.to_csv('Desktop/Comfort_zone_paradigm/data/NEWSpider.csv')
spider['locomotion'] = listsumlocomotion
spider.to_csv('data/NEWSpider.csv')

In [25]:
print error

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(error)? (<ipython-input-25-bff49b1d4746>, line 1)

In [ ]:
dfc = pd.concat(list_df)
dfc = dfc[dfc.blue == 0]
dfc = dfc[dfc.Recording_time>120]
dfc = dfc[dfc.Recording_time<180]
dfc = dfc[dfc.X_center > 8.32]
dfc = dfc[dfc.trial == 110]
x = dfc.X_center
z = dfc.Z_center

ax = sns.kdeplot(x, z, cmap="coolwarm_r", shade=True, n_levels=30, shade_lowest=False, cbar=True)

#ax = sns.kdeplot(x, z, cmap="coolwarm")

# the function has additional parameters you can play around with to fine-tune your heatmap, e.g.:
#ax = sns.kdeplot(x, z, kernel="gau", bw = 25, cmap="coolwarm", n_levels = 50, shade=True, shade_lowest=False, gridsize=100)

# plot your KDE
#ax.set_frame_on(False)
plt.xlim(27, 5)
plt.ylim(-6, 11)
#plt.axis('off')
plt.show()

# save your KDE to disk
fig = ax.get_figure()
#fig.savefig('Desktop/Comfort_zone_paradigm/kde.png', transparent=True, bbox_inches='tight', pad_inches=0)
#fig.savefig('Desktop/Comfort_zone_paradigm/kde.png', dpi=1000)
fig.savefig('kde.png', dpi=1000)

In [ ]:
listi=[]
for i in range(0, len(spider.int_number)):
        if spider.date[i] == '10.05.2020' or spider.date[i] == '30.05.2020'or spider.date[i] == '14.06.2020'or spider.date[i] == '24.06.2020'or spider.date[i] == '08.07.2020'or spider.date[i] == '14.07.2020':
            if spider.Strain[i] == 'Pomc:bPAC+' and spider.Treatment[i] =='None':
                if spider.int_number[i] < 30: 
                    listi.append('+')
                else:
                    listi.append('-')
            else:
                listi.append('none')
    
        else:
            listi.append('none')
        
p=0
for df1 in list_df:
    df1['phenotype'] = listi[p]
    p=p+1
            

In [ ]:
import matplotlib.patches as patches

dfc = pd.concat(list_df)
#dfc = dfc[dfc.blue == 0]
dfc = dfc[dfc.Recording_time>120]
dfc = dfc[dfc.Recording_time<180]
#dfc = dfc[dfc.X_center > 8.32]
#dfc = dfc[dfc.trial == 110]
#dfc = dfc[dfc.strain == 'TU']
dfc = dfc[dfc.phenotype == '+']
x = dfc.X_center
z = dfc.Z_center

f, (ax1, ax2) = plt.subplots(2, 1, sharex=True);
ax1.plot(x, z, linewidth=0.3, zorder=5, color = 'grey');
ax2 = sns.kdeplot(x, z, cmap="coolwarm", n_levels=30, shade=True, shade_lowest=False)   

ax1.set_aspect(1.0)
ax2.set_aspect(1.0)

ax1.set_frame_on(False)
plt.axis('off')
ax2.set_frame_on(False)

#ax1.set_xlim(25, 0);
#ax1.set_ylim(13.2, -4.2);
#ax2.set_xlim(25, 0);
#ax2.set_ylim(13.2, -4.2);

ax1.set_xlim(27, -1);
ax1.set_ylim(14.2, -5.2);
ax2.set_xlim(27, -1);
ax2.set_ylim(14.2, -5.2);

ax1.set_yticklabels([])
ax1.set_xticklabels([])
ax2.set_yticklabels([])
ax2.set_xticklabels([])

ax1.xaxis.label.set_visible(False)
ax2.xaxis.label.set_visible(False)
ax1.yaxis.label.set_visible(False)
ax2.yaxis.label.set_visible(False)


colour='royalblue'

#  recatangle ==  (x start, y start), x len, y len

#rect=patches.Rectangle((0,-4.2),8.4,13.2,linewidth=1,edgecolor=colour,facecolor=colour, zorder=25)
#ax1.add_patch(rect)

#rect=patches.Rectangle((0,-4.2),8.4,13.2,linewidth=1,edgecolor=colour,facecolor=colour, zorder=25)
#ax2.add_patch(rect)
#((0,-5),8.4,15
#rect=patches.Rectangle((0,-4.2),25,13.2,linewidth=1,edgecolor='black',facecolor='none', zorder=35)
#ax1.add_patch(rect)

#rect=patches.Rectangle((0,-4.2),25,13.2,linewidth=1,edgecolor='black',facecolor='none', zorder=35)
#ax2.add_patch(rect)


#f.savefig('Desktop/Comfort_zone_paradigm/testtu', dpi=1000);
f.savefig('testtu', dpi=1000);


# Traces

In [ ]:
list(df1.columns.values)

In [ ]:
ranges=(max(df1.Z_center))-(min(df1.Z_center))
half=(min(df1.Z_center))+(ranges/2)
print(ranges)
print(half)
print((min(df1.Z_center)))

In [ ]:
sns.set_style(style='white')


list_df2=[]

for df1 in list_df:
    list_df2.append(df1)
    
    
listf,o,p=[],1,0
for df2 in list_df2:
    listx,listx2,listy,listy2 = [],[],[],[]
    for i in range(0, len(df2.X_center)):
        if df2.Recording_time[i]<Time_divider_pulled and df2.Recording_time[i]>0:
            if df2.Distance_moved[i]<1.1:
                if df2.Distance_moved[i] > 0:
                    x = df2.Recording_time[i]
                    y = df2.Z_center[i]
                    #x = df1.X_center_side[i]
                    #y = df1.Y_center[i]
                    listx.append(x)
                    listy.append(y)
                else:
                    pass     
        elif df2.Recording_time[i]>Time_divider_pulled:
            if df2.Distance_moved[i]<1.1:
                if df2.Distance_moved[i] > 0:
                    if df2.interaction_zone[i] == 1:
                        x2 = df2.Recording_time[i]
                        y2 = df2.Z_center[i]
                        listx2.append(x2)
                        listy2.append(y2)
                    else:
                        x2 = df2.Recording_time[i]
                        y2 = 2.5
                        listx2.append(x2)
                        listy2.append(y2)
        
    f, (ax1, ax2) = plt.subplots(2, 1, sharex=True);
    ax1.plot(listx, listy, linewidth=0.5, color="#79B8C4");
    ax1.set_title(p);
    ax2.plot(listx2, listy2, linewidth=0.5, color="#79B8C4");
    
        
    #ax1.set_xlim(1, 24);
    #ax1.set_ylim(-4.5, 4);
    #ax2.set_xlim(1, 24);
    #ax2.set_ylim(-4.5, 4);
    
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax1.grid(False)
    ax1.get_xaxis().set_visible(False)
    ax2.get_xaxis().set_visible(False)
    ax1.set_title('Subject '+str(int(spider.Subject[p]))+' Interactions: '+str(int(spider.int_number[p]))+' strain: '+str((spider.Strain[p])));
    if p ==3 or p == 4:
        f.savefig('Traces/Trial_'+str(o)+'.png', dpi=2000);

    o,p=o+1,p+1
    listf.append(f)

In [ ]:
listf,o,p=[],1,0
for df1 in list_df:
    listx,listx2,listy,listy2 = [],[],[],[]
    for i in range(0, len(df1.X_center)):
        if df1.Recording_time[i]<Time_divider_pulled and df1.Recording_time[i]>0:
            if df1.Distance_moved[i]<1.1:
                if df1.Distance_moved[i] > 0:
                    x = df1.Recording_time[i]
                    y = df1.Z_center[i]
                    #x = df1.X_center_side[i]
                    #y = df1.Y_center[i]
                    listx.append(x)
                    listy.append(y)
                else:
                    pass     
        elif df1.Recording_time[i]>Time_divider_pulled:
            if df1.Distance_moved[i]<1.1:
                if df1.Distance_moved[i] > 0:
                    x2 = df1.Recording_time[i]
                    y2 = df1.Z_center[i]
                    listx2.append(x2)
                    listy2.append(y2)
                else:
                    pass
        
    f, (ax1, ax2) = plt.subplots(2, 1, sharex=True);
    ax1.plot(listx, listy, linewidth=0.5);
    ax1.set_title(p);
    ax2.plot(listx2, listy2, linewidth=0.5);
    
    #ax1.set_xlim(1, 24);
    #ax1.set_ylim(-4.5, 4);
    #ax2.set_xlim(1, 24);
    #ax2.set_ylim(-4.5, 4);
    
    ax1.set_title('Subject '+str(int(spider.Subject[p]))+' Social zone: '+str(int(table.social[p]))+'  White zone: '+str(int(table.white[p]))+'  Blue zone: '+str(int(table.blue[p]))+'       ');
    ax2.set_title('Subject '+str(spider.Subject[p])+' Social zone: '+str(int(table2.social[p]))+'  White zone: '+str(int(table2.white[p]))+'  Blue zone: '+str(int(table2.blue[p]))+'       ');
    
    f.savefig('Traces/Trial_'+str(o)+'.png', dpi=1000);
    o,p=o+1,p+1
    listf.append(f)

In [ ]:
listf,o,p=[],1,0
for df1 in list_df:
    listx,listx2,listy,listy2 = [],[],[],[]
    for i in range(0, len(df1.X_center)):
        if df1.Recording_time[i]<Time_divider_pulled and df1.Recording_time[i]>0:
            if df1.Distance_moved[i]<1.1:
                if df1.Distance_moved[i] > 0:
                    #x = df1.X_center_side[i]
                    #y = df1.Y_center[i]
                    x = df1.X_center[i]
                    y = df1.Z_center[i]
                    listx.append(x)
                    listy.append(y)
                else:
                    pass     
        elif df1.Recording_time[i]>Time_divider_pulled:
            if df1.Distance_moved[i]<1.1:
                if df1.Distance_moved[i] > 0:
                    #x2 = df1.X_center_side[i]
                    #y2 = df1.Y_center[i]
                    x2 = df1.X_center[i]
                    y2 = df1.Z_center[i]
                    listx2.append(x2)
                    listy2.append(y2)
                else:
                    pass
        
    f, (ax1, ax2) = plt.subplots(2, 1, sharex=True);
    ax1.plot(listx, listy, linewidth=0.5);
    ax1.set_title(p);
    ax2.plot(listx2, listy2, linewidth=0.5);
    
    #ax1.set_xlim(1, 24);
    #ax1.set_ylim(-4.5, 4);
    #ax2.set_xlim(1, 24);
    #ax2.set_ylim(-4.5, 4);
    
    ax1.set_title('Subject '+str(int(spider.Subject[p]))+' Social zone: '+str(int(table.social[p]))+'  White zone: '+str(int(table.white[p]))+'  Blue zone: '+str(int(table.blue[p]))+'       ');
    ax2.set_title('Subject '+str(spider.Subject[p])+' Social zone: '+str(int(table2.social[p]))+'  White zone: '+str(int(table2.white[p]))+'  Blue zone: '+str(int(table2.blue[p]))+'       ');
    
    #f.savefig('Desktop/Comfort_zone_paradigm/data/Traces/Trial_'+str(o)+'.png', dpi=1000);
    f.savefig('Traces/Trial_'+str(o)+'.png', dpi=1000);
    o,p=o+1,p+1
    listf.append(f)

In [ ]:
#side

In [ ]:
listf,o,p=[],1,0
for df1 in list_df:
    listx,listx2,listy,listy2 = [],[],[],[]
    for i in range(0, len(df1.X_center)):
        if df1.Recording_time[i]<Time_divider_pulled and df1.Recording_time[i]>0:
            if df1.Distance_moved[i]<1.1:
                if df1.Distance_moved[i] > 0:
                    #x = df1.X_center_side[i]
                    #y = df1.Y_center[i]
                    x = df1.X_center[i]
                    y = df1.Z_center[i]
                    listx.append(x)
                    listy.append(y)
                else:
                    pass     
        elif df1.Recording_time[i]>Time_divider_pulled:
            if df1.Distance_moved[i]<1.1:
                if df1.Distance_moved[i] > 0:
                    #x2 = df1.X_center_side[i]
                    #y2 = df1.Y_center[i]
                    x2 = df1.X_center[i]
                    y2 = df1.Z_center[i]
                    listx2.append(x2)
                    listy2.append(y2)
                else:
                    pass
        
    f, (ax1, ax2) = plt.subplots(2, 1, sharex=True);
    ax1.plot(listx, listy, linewidth=0.5);
    ax1.set_title(p);
    ax2.plot(listx2, listy2, linewidth=0.5);
    
    #ax1.set_xlim(1, 24);
    #ax1.set_ylim(-4.5, 4);
    #ax2.set_xlim(1, 24);
    #ax2.set_ylim(-4.5, 4);
    
    ax1.set_title('Subject '+str(int(spider.Subject[p]))+' Social zone: '+str(int(table.social[p]))+'  White zone: '+str(int(table.white[p]))+'  Blue zone: '+str(int(table.blue[p]))+'       ');
    ax2.set_title('Subject '+str(spider.Subject[p])+' Social zone: '+str(int(table2.social[p]))+'  White zone: '+str(int(table2.white[p]))+'  Blue zone: '+str(int(table2.blue[p]))+'       ');
    
    #f.savefig('Desktop/Comfort_zone_paradigm/data/Traces/Trial_'+str(o)+'.png', dpi=1000);
    o,p=o+1,p+1
    listf.append(f)

# Final individual spider plot

In [ ]:
experiment = 'all'
basalortest= 'Basal'


spiderbasal = spider    
spiderbasal = spiderbasal[spiderbasal.Condition == 'Basal']
spiderbasal = spiderbasal[spiderbasal.Strain != 'Pomc:bPAC+']
spiderbasal = spiderbasal[spiderbasal.Strain != 'Pomc:bPAC-']
spiderbasal = spiderbasal[spiderbasal.date != '11.04.2020']
spiderbasal = spiderbasal[spiderbasal.Strain == 'TU']
spiderbasal = spiderbasal[spiderbasal.Treatment == 'None']
spiderbasal = spiderbasal[spiderbasal.interaction_zone > 18]

spiderbasal = spiderbasal.reset_index(drop=True)
print(len(spiderbasal))


rejects=[]
if basalortest == 'Test':
    for i in range(0,len(listtrialprev)):
        for p in range(0,len(spiderbasal.Subject)):
            if spiderbasal.Subject[p] == listtrialprev[i] and spiderbasal.date[p] == listdateprev[i]:
                rejects.append(p)
                #print(p)
            else:
                pass
                
else:
    pass



if basalortest == 'Test':   
    spiderbasal = spiderbasal.ix[rejects]  
else:
    pass  
    
        
spiderbasal = spiderbasal.reset_index(drop=True)
listtrial,list_latency,list_break_duration,listdate=[],[],[],[]
list_blue_pre0,list_blue_pre1,list_blue_pre2,list_blue_pre4,list_social0,list_social1,list_social2,list_social4,list_shoal0,list_shoal1,list_shoal2,list_shoal4=[],[],[],[],[],[],[],[],[],[],[],[]
list_blue0,list_blue1,list_blue2,list_blue4,list_depth_range0,list_depth_range1,list_depth_range2,list_depth_range4,list_depth_mean0,list_depth_mean1,list_depth_mean2,list_depth_mean4=[],[],[],[],[],[],[],[],[],[],[],[]
list_interaction_frequency0,list_interaction_frequency1,list_interaction_frequency2,list_interaction_frequency4,list_int_duration0,list_int_duration1,list_int_duration2,list_int_duration4=[],[],[],[],[],[],[],[]
list_int_speed0,list_int_speed1,list_int_speed2,list_int_speed4=[],[],[],[]
list_interaction_zone, list_freeze = [],[]
for i in range(0,len(spiderbasal.blue)):
        list_blue0.append(100-spiderbasal.blue[i])
        list_freeze.append(100-spiderbasal.freeze[i])
        list_blue_pre0.append(spiderbasal.blue_pre[i]) 
        list_social0.append(spiderbasal.social[i]) 
        list_shoal0.append(spiderbasal.shoal[i]) 
        list_depth_range0.append(spiderbasal.depth_range[i]-2) 
        list_depth_mean0.append(15*(spiderbasal.depth_mean[i]))
        list_interaction_frequency0.append(1.25*(spiderbasal.int_number[i]))


        list_latency.append(101-((spiderbasal.Latency[i]-60)*0.84))

        listtrial.append(spiderbasal.Subject[i])
        listdate.append(spiderbasal.date[i])
        list_interaction_zone.append(spiderbasal.latestage_interaction[i])
        
if basalortest == 'Basal':
    listtrialprev=listtrial
    listdateprev=listdate
else:
    pass
        
df = pd.DataFrame({
    'Blue preference':list_blue_pre0,
    'Freezing':list_freeze,
    'Interaction zone':list_interaction_zone,
    
    'Interaction count':list_interaction_frequency0,
    
    'Latency':list_latency,
    #'Locomotion':list_break_duration,
    
    })

# number of variable

categories=list(df)[0:]
N = len(categories)

 
# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

# Initialise the spider plot
ax = plt.subplot(111, polar=True)
 
# If you want the first axis to be on top:
ax.set_theta_offset(pi / 2)
ax.set_theta_direction(-1)
 
# Draw one axe per variable + add labels labels yet
plt.xticks(angles[:-1], categories)
 
# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([20,40,60,80,100], ["20",'40','60','80','100'], color="grey", size=7)
plt.ylim(0,100)

for i in range(0,len(spiderbasal.blue)):
    values = df.loc[i].values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=1, linestyle='solid', label= (str(listtrial[i])+' - '+str(listdate[i])))

#plt.title(experiment, y=1.05, x=-0.1)

#plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1.1));
#plt.legend(loc='upper right', bbox_to_anchor=(1.6, 1.1));
plt.savefig('Desktop/Comfort_zone_paradigm/data/spider_Female.png', dpi=300);

print(len(spiderbasal))

In [ ]:
spidertableshow = spider
spidertableshow = spidertableshow.filter(['Condition','Treatment','Subject','Strain'], axis=1) 
print(spidertableshow)

# Final group spider plot

In [ ]:
filter_option = 'Treatment'

listfilterout=[]
for i in range(0,len(spider.blue)):
    if spider.Condition[i] == 'Basal' and spider.interaction_zone[i] < 20:
        filterout = 'yes'
    else:
        filterout = 'no'
    listfilterout.append(filterout)
        
spider['filterout'] =  listfilterout   

spiderbasal = spider
spiderbasal = spiderbasal[spiderbasal.Strain == 'TU']
spiderbasal = spiderbasal[spiderbasal.Treatment != 3]
spiderbasal = spiderbasal[spiderbasal.Treatment != 24]
spiderbasal = spiderbasal[spiderbasal.Treatment != '2']
spiderbasal = spiderbasal[spiderbasal.date != '11.04.2020']
spiderbasal = spiderbasal[spiderbasal.Treatment != '8']
spiderbasal = spiderbasal[spiderbasal.filterout =='no']

spiderbasal = spiderbasal.reset_index(drop=True)

spiderbasal['Treatment'] = spiderbasal['Treatment'].astype(str)
    
    
spiderbasal.to_csv('Desktop/Comfort_zone_paradigm/data/spiderbasal.csv');


grouped = (spiderbasal.groupby('Treatment').groups)


listgroup=[]

for name in grouped:
    group=name
    listgroup.append(group)


    
list_blue_pre0,list_blue_pre1,list_blue_pre2,list_blue_pre3,list_social0,list_social1,list_social2,list_social3,list_shoal0,list_shoal1,list_shoal2,list_shoal3=[],[],[],[],[],[],[],[],[],[],[],[]
list_blue0,list_blue1,list_blue2,list_blue3,list_depth_range0,list_depth_range1,list_depth_range2,list_depth_range3,list_depth_mean0,list_depth_mean1,list_depth_mean2,list_depth_mean3=[],[],[],[],[],[],[],[],[],[],[],[]
list_interaction_frequency0,list_interaction_frequency1,list_interaction_frequency2,list_interaction_frequency3,list_int_duration0,list_int_duration1,list_int_duration2,list_int_duration3=[],[],[],[],[],[],[],[]
list_int_speed0,list_int_speed1,list_int_speed2,list_int_speed3=[],[],[],[]
list_interaction0,list_interaction1,list_interaction2,list_interaction3 = [],[],[],[]
list_latency0,list_latency1,list_latency2,list_latency3,list_latency4 =[],[],[],[],[]
list_freeze0,list_freeze1,list_freeze2,list_freeze3,list_freeze4 =[],[],[],[],[]
list_interaction_frequency0,list_interaction_frequency1,list_interaction_frequency2,list_interaction_frequency3,list_interaction_frequency4 =[],[],[],[],[]
list_interaction0_zone,list_interaction1_zone,list_interaction2_zone,list_interaction3_zone,list_interaction4_zone = [],[],[],[],[]
list_blue_pre4, list_interaction_frequency4=[],[]

for i in range(0,len(spiderbasal.blue)):
    if spiderbasal.Treatment[i]==listgroup[0]:
        list_blue_pre0.append(spiderbasal.blue_pre[i]) 
        list_freeze0.append(100-spiderbasal.freeze[i])
        list_interaction0_zone.append(spiderbasal.interaction_zone[i])
        list_latency0.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency0.append(1.25*(spiderbasal.int_number[i]))
    if spiderbasal.Treatment[i]==listgroup[1]: 
        list_blue_pre1.append(spiderbasal.blue_pre[i]) 
        list_freeze1.append(100-spiderbasal.freeze[i])
        list_interaction1_zone.append(spiderbasal.interaction_zone[i])
        list_latency1.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency1.append(1.25*(spiderbasal.int_number[i]))
    if spiderbasal.Treatment[i]==listgroup[2]:
        list_blue_pre2.append(spiderbasal.blue_pre[i]) 
        list_freeze2.append(100-spiderbasal.freeze[i])
        list_interaction2_zone.append(spiderbasal.interaction_zone[i])
        list_latency2.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency2.append(1.25*(spiderbasal.int_number[i]))
    if spiderbasal.Treatment[i]==listgroup[3]:
        list_blue_pre3.append(spiderbasal.blue_pre[i]) 
        list_freeze3.append(100-spiderbasal.freeze[i])
        list_interaction3_zone.append(spiderbasal.interaction_zone[i])
        list_latency3.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency3.append(1.25*(spiderbasal.int_number[i]))
    if spiderbasal.Treatment[i]==listgroup[4]:
        list_blue_pre4.append(spiderbasal.blue_pre[i]) 
        list_freeze4.append(100-spiderbasal.freeze[i])
        list_interaction4_zone.append(spiderbasal.interaction_zone[i])
        list_latency4.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency4.append(0.75*(spiderbasal.int_number[i]))


blue_preav0 = (sum(list_blue_pre0)/len(list_blue_pre0))
blue_preav1 = (sum(list_blue_pre1)/len(list_blue_pre1))   
blue_preav2 = (sum(list_blue_pre2)/len(list_blue_pre2)) 
blue_preav3 = (sum(list_blue_pre3)/len(list_blue_pre3)) 
blue_preav4 = (sum(list_blue_pre4)/len(list_blue_pre4)) 

interaction_frequencyav0 = (sum(list_interaction_frequency0)/len(list_interaction_frequency0))
interaction_frequencyav1 = (sum(list_interaction_frequency1)/len(list_interaction_frequency1))
interaction_frequencyav2 = (sum(list_interaction_frequency2)/len(list_interaction_frequency2))
interaction_frequencyav3 = (sum(list_interaction_frequency3)/len(list_interaction_frequency3))
interaction_frequencyav4 = (sum(list_interaction_frequency4)/len(list_interaction_frequency4))

freezeyav0 = (sum(list_freeze0)/len(list_freeze0))
freezeyav1 = (sum(list_freeze1)/len(list_freeze1))
freezeyav2 = (sum(list_freeze2)/len(list_freeze2))
freezeyav3 = (sum(list_freeze3)/len(list_freeze3))
freezeyav4 = (sum(list_freeze4)/len(list_freeze4))

latencyav0 = (sum(list_latency0)/len(list_latency0))
latencyav1 = (sum(list_latency1)/len(list_latency1))
latencyav2 = (sum(list_latency2)/len(list_latency2))
latencyav3 = (sum(list_latency3)/len(list_latency3))
latencyav4 = (sum(list_latency4)/len(list_latency4))

interactionav0 = (sum(list_interaction0_zone)/len(list_interaction0_zone))
interactionav1 = (sum(list_interaction1_zone)/len(list_interaction1_zone))
interactionav2 = (sum(list_interaction2_zone)/len(list_interaction2_zone))
interactionav3 = (sum(list_interaction3_zone)/len(list_interaction3_zone))
interactionav4 = (sum(list_interaction4_zone)/len(list_interaction4_zone))


df = pd.DataFrame({
    'group': [str(listgroup[0]),str(listgroup[1]),str(listgroup[2]),str(listgroup[3]),str(listgroup[4])],
    'Blue preference':[blue_preav0,blue_preav1,blue_preav2,blue_preav3,blue_preav4],
    'Freezing':[freezeyav0,freezeyav1,freezeyav2,freezeyav3,freezeyav4],
    'Interaction zone':[interactionav0,interactionav1,interactionav2,interactionav3,interactionav4],
    'Interaction count':[interaction_frequencyav0,interaction_frequencyav1,interaction_frequencyav2,interaction_frequencyav3,interaction_frequencyav4],
    'Latency':[latencyav0,latencyav1,latencyav2,latencyav3,latencyav4]
    })


# number of variable
categories=list(df)[1:]
N = len(categories)
 
# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]
 
# Initialise the spider plot
ax = plt.subplot(111, polar=True)
 
# If you want the first axis to be on top:
ax.set_theta_offset(pi / 2)
ax.set_theta_direction(-1)
 
# Draw one axe per variable + add labels labels yet
plt.xticks(angles[:-1], categories)
 
# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([20,40,60,80,100], ["20",'40','60','80','100'], color="grey", size=7)
plt.ylim(0,100)
 
    
# Ind4
values=df.loc[4].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label='TU WT')
ax.fill(angles, values, 'r', alpha=0.00)
    
# Ind1
values=df.loc[0].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label= str(listgroup[0]))
ax.fill(angles, values, 'b', alpha=0.0)

# Ind3
values=df.loc[3].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label=str(listgroup[3]))
ax.fill(angles, values, 'r', alpha=0.00)

 
# Ind2
values=df.loc[1].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label=str(listgroup[1]))
ax.fill(angles, values, 'r', alpha=0.00)

# Ind3
values=df.loc[2].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label=str(listgroup[2]))
ax.fill(angles, values, 'r', alpha=0.000)
              


# Add legend
plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1.12));

plt.savefig('Desktop/Comfort_zone_paradigm/data/spider_stress_matrix.png', dpi=1000);

In [ ]:
filter_option = 'Treatment'

listfilterout=[]
for i in range(0,len(spider.blue)):
    if spider.Condition[i] == 'Basal' and spider.interaction_zone[i] < 20:
        filterout = 'yes'
    else:
        filterout = 'no'
    listfilterout.append(filterout)
        
spider['filterout'] =  listfilterout   

spiderbasal = spider
spiderbasal = spiderbasal[spiderbasal.Strain == 'TU']
spiderbasal = spiderbasal[spiderbasal.Treatment != 3]
spiderbasal = spiderbasal[spiderbasal.Treatment != 24]
spiderbasal = spiderbasal[spiderbasal.Treatment != '2']
spiderbasal = spiderbasal[spiderbasal.date != '11.04.2020']
spiderbasal = spiderbasal[spiderbasal.Treatment != '8']
spiderbasal = spiderbasal[spiderbasal.filterout =='no']

spiderbasal = spiderbasal.reset_index(drop=True)

spiderbasal['Treatment'] = spiderbasal['Treatment'].astype(str)
    
    
spiderbasal.to_csv('Desktop/Comfort_zone_paradigm/data/spiderbasal.csv');


grouped = (spiderbasal.groupby('Treatment').groups)


listgroup=[]

for name in grouped:
    group=name
    listgroup.append(group)


    
list_blue_pre0,list_blue_pre1,list_blue_pre2,list_blue_pre3,list_social0,list_social1,list_social2,list_social3,list_shoal0,list_shoal1,list_shoal2,list_shoal3=[],[],[],[],[],[],[],[],[],[],[],[]
list_blue0,list_blue1,list_blue2,list_blue3,list_depth_range0,list_depth_range1,list_depth_range2,list_depth_range3,list_depth_mean0,list_depth_mean1,list_depth_mean2,list_depth_mean3=[],[],[],[],[],[],[],[],[],[],[],[]
list_interaction_frequency0,list_interaction_frequency1,list_interaction_frequency2,list_interaction_frequency3,list_int_duration0,list_int_duration1,list_int_duration2,list_int_duration3=[],[],[],[],[],[],[],[]
list_int_speed0,list_int_speed1,list_int_speed2,list_int_speed3=[],[],[],[]
list_interaction0,list_interaction1,list_interaction2,list_interaction3 = [],[],[],[]
list_latency0,list_latency1,list_latency2,list_latency3,list_latency4 =[],[],[],[],[]
list_freeze0,list_freeze1,list_freeze2,list_freeze3,list_freeze4 =[],[],[],[],[]
list_interaction_frequency0,list_interaction_frequency1,list_interaction_frequency2,list_interaction_frequency3,list_interaction_frequency4 =[],[],[],[],[]
list_interaction0_zone,list_interaction1_zone,list_interaction2_zone,list_interaction3_zone,list_interaction4_zone = [],[],[],[],[]
list_blue_pre4, list_interaction_frequency4=[],[]

for i in range(0,len(spiderbasal.blue)):
    if spiderbasal.Treatment[i]==listgroup[0]:
        list_blue_pre0.append(spiderbasal.blue_pre[i]) 
        list_freeze0.append(100-spiderbasal.freeze[i])
        list_interaction0_zone.append(spiderbasal.interaction_zone[i])
        list_latency0.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency0.append(1.25*(spiderbasal.int_number[i]))
    if spiderbasal.Treatment[i]==listgroup[1]: 
        list_blue_pre1.append(spiderbasal.blue_pre[i]) 
        list_freeze1.append(100-spiderbasal.freeze[i])
        list_interaction1_zone.append(spiderbasal.interaction_zone[i])
        list_latency1.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency1.append(1.25*(spiderbasal.int_number[i]))
    if spiderbasal.Treatment[i]==listgroup[2]:
        list_blue_pre2.append(spiderbasal.blue_pre[i]) 
        list_freeze2.append(100-spiderbasal.freeze[i])
        list_interaction2_zone.append(spiderbasal.interaction_zone[i])
        list_latency2.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency2.append(1.25*(spiderbasal.int_number[i]))
    if spiderbasal.Treatment[i]==listgroup[3]:
        list_blue_pre3.append(spiderbasal.blue_pre[i]) 
        list_freeze3.append(100-spiderbasal.freeze[i])
        list_interaction3_zone.append(spiderbasal.interaction_zone[i])
        list_latency3.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency3.append(1.25*(spiderbasal.int_number[i]))
    if spiderbasal.Treatment[i]==listgroup[4]:
        list_blue_pre4.append(spiderbasal.blue_pre[i]) 
        list_freeze4.append(100-spiderbasal.freeze[i])
        list_interaction4_zone.append(spiderbasal.interaction_zone[i])
        list_latency4.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency4.append(0.75*(spiderbasal.int_number[i]))


blue_preav0 = (sum(list_blue_pre0)/len(list_blue_pre0))
blue_preav1 = (sum(list_blue_pre1)/len(list_blue_pre1))   
blue_preav2 = (sum(list_blue_pre2)/len(list_blue_pre2)) 
blue_preav3 = (sum(list_blue_pre3)/len(list_blue_pre3)) 
blue_preav4 = (sum(list_blue_pre4)/len(list_blue_pre4)) 

interaction_frequencyav0 = (sum(list_interaction_frequency0)/len(list_interaction_frequency0))
interaction_frequencyav1 = (sum(list_interaction_frequency1)/len(list_interaction_frequency1))
interaction_frequencyav2 = (sum(list_interaction_frequency2)/len(list_interaction_frequency2))
interaction_frequencyav3 = (sum(list_interaction_frequency3)/len(list_interaction_frequency3))
interaction_frequencyav4 = (sum(list_interaction_frequency4)/len(list_interaction_frequency4))

freezeyav0 = (sum(list_freeze0)/len(list_freeze0))
freezeyav1 = (sum(list_freeze1)/len(list_freeze1))
freezeyav2 = (sum(list_freeze2)/len(list_freeze2))
freezeyav3 = (sum(list_freeze3)/len(list_freeze3))
freezeyav4 = (sum(list_freeze4)/len(list_freeze4))

latencyav0 = (sum(list_latency0)/len(list_latency0))
latencyav1 = (sum(list_latency1)/len(list_latency1))
latencyav2 = (sum(list_latency2)/len(list_latency2))
latencyav3 = (sum(list_latency3)/len(list_latency3))
latencyav4 = (sum(list_latency4)/len(list_latency4))

interactionav0 = (sum(list_interaction0_zone)/len(list_interaction0_zone))
interactionav1 = (sum(list_interaction1_zone)/len(list_interaction1_zone))
interactionav2 = (sum(list_interaction2_zone)/len(list_interaction2_zone))
interactionav3 = (sum(list_interaction3_zone)/len(list_interaction3_zone))
interactionav4 = (sum(list_interaction4_zone)/len(list_interaction4_zone))


df = pd.DataFrame({
    'group': [str(listgroup[0]),str(listgroup[1]),str(listgroup[2]),str(listgroup[3]),str(listgroup[4])],
    'Blue preference':[blue_preav0,blue_preav1,blue_preav2,blue_preav3,blue_preav4],
    'Freezing':[freezeyav0,freezeyav1,freezeyav2,freezeyav3,freezeyav4],
    'Interaction zone':[interactionav0,interactionav1,interactionav2,interactionav3,interactionav4],
    'Interaction count':[interaction_frequencyav0,interaction_frequencyav1,interaction_frequencyav2,interaction_frequencyav3,interaction_frequencyav4],
    'Latency':[latencyav0,latencyav1,latencyav2,latencyav3,latencyav4]
    })


# number of variable
categories=list(df)[1:]
N = len(categories)
 
# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]
 
# Initialise the spider plot
ax = plt.subplot(111, polar=True)
 
# If you want the first axis to be on top:
ax.set_theta_offset(pi / 2)
ax.set_theta_direction(-1)
 
# Draw one axe per variable + add labels labels yet
plt.xticks(angles[:-1], categories)
 
# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([20,40,60,80,100], ["20",'40','60','80','100'], color="grey", size=7)
plt.ylim(0,100)
 
    
# Ind4
values=df.loc[4].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label='TU WT')
ax.fill(angles, values, 'r', alpha=0.00)
    
# Ind1
values=df.loc[0].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label= str(listgroup[0]))
ax.fill(angles, values, 'b', alpha=0.0)

# Ind3
values=df.loc[3].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label=str(listgroup[3]))
ax.fill(angles, values, 'r', alpha=0.00)

 
# Ind2
values=df.loc[1].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label=str(listgroup[1]))
ax.fill(angles, values, 'r', alpha=0.00)

# Ind3
values=df.loc[2].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label=str(listgroup[2]))
ax.fill(angles, values, 'r', alpha=0.000)
              


# Add legend
plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1.12));

plt.savefig('Desktop/Comfort_zone_paradigm/data/spider_star_matrix.png', dpi=600);

Pomc:bPAC Spider

In [ ]:
filter_option = 'Treatment'

listfilterout=[]
for i in range(0,len(spider.blue)):
    if spider.Condition[i] == 'Basal' and spider.Strain[i] == 'TU' and spider.interaction_zone[i] < 20:
        filterout = 'yes'
    else:
        filterout = 'no'
    listfilterout.append(filterout)
        
spider['filterout'] =  listfilterout   


spiderbasal = spider
spiderbasal = spiderbasal[spiderbasal.filterout =='no']
spiderbasal = spiderbasal[spiderbasal.Condition == 'Basal']
spiderbasal = spiderbasal[spiderbasal.Strain != 'TU Male']

#spiderbasal = spiderbasal[spider.date != '14.03.2020']
spiderbasal = spiderbasal[spiderbasal.date != '11.04.2020']
spiderbasal = spiderbasal[spiderbasal.date != '12.04.2020']
spiderbasal = spiderbasal[spiderbasal.Strain != 'Star:bPAC-']
spiderbasal = spiderbasal[spiderbasal.Strain != 'Star:bPAC+']
spiderbasal = spiderbasal[spiderbasal.Treatment == 'None']

print(len(spiderbasal))
spiderbasal = spiderbasal.reset_index(drop=True)

grouped = (spiderbasal.groupby('Strain').groups)
#grouped = (spiderbasal.groupby('Treatment').groups)

listgroup=[]

for name in grouped:
    group=name
    listgroup.append(group)

print(listgroup)
    
list_blue_pre0,list_blue_pre1,list_blue_pre2,list_blue_pre3,list_social0,list_social1,list_social2,list_social3,list_shoal0,list_shoal1,list_shoal2,list_shoal3=[],[],[],[],[],[],[],[],[],[],[],[]
list_blue0,list_blue1,list_blue2,list_blue3,list_depth_range0,list_depth_range1,list_depth_range2,list_depth_range3,list_depth_mean0,list_depth_mean1,list_depth_mean2,list_depth_mean3=[],[],[],[],[],[],[],[],[],[],[],[]
list_interaction_frequency0,list_interaction_frequency1,list_interaction_frequency2,list_interaction_frequency3,list_int_duration0,list_int_duration1,list_int_duration2,list_int_duration3=[],[],[],[],[],[],[],[]
list_int_speed0,list_int_speed1,list_int_speed2,list_int_speed3=[],[],[],[]
list_interaction0,list_interaction1,list_interaction2,list_interaction3 = [],[],[],[]
list_latency0,list_latency1,list_latency2,list_latency3,list_latency4 =[],[],[],[],[]
list_freeze0,list_freeze1,list_freeze2,list_freeze3,list_freeze4 =[],[],[],[],[]
list_interaction_frequency0,list_interaction_frequency1,list_interaction_frequency2,list_interaction_frequency3,list_interaction_frequency4 =[],[],[],[],[]
list_interaction0_zone,list_interaction1_zone,list_interaction2_zone,list_interaction3_zone,list_interaction4_zone = [],[],[],[],[]
list_blue_pre4, list_interaction_frequency4=[],[]

for i in range(0,len(spiderbasal.blue)):
    if spiderbasal.Strain[i]==listgroup[0]:
        list_blue_pre0.append(spiderbasal.blue_pre[i]) 
        list_freeze0.append(100-spiderbasal.freeze[i])
        list_interaction0_zone.append(spiderbasal.latestage_interaction[i])
        list_latency0.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency0.append(0.75*(spiderbasal.int_number[i]))
    if spiderbasal.Strain[i]==listgroup[1]: 
        list_blue_pre1.append(spiderbasal.blue_pre[i]) 
        list_freeze1.append(100-spiderbasal.freeze[i])
        list_interaction1_zone.append(spiderbasal.latestage_interaction[i])
        list_latency1.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency1.append(0.75*(spiderbasal.int_number[i]))
    if spiderbasal.Strain[i]==listgroup[2]:
        list_blue_pre2.append(spiderbasal.blue_pre[i]) 
        list_freeze2.append(100-spiderbasal.freeze[i])
        list_interaction2_zone.append(spiderbasal.latestage_interaction[i])
        list_latency2.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency2.append(0.75*(spiderbasal.int_number[i]))


blue_preav0 = (sum(list_blue_pre0)/len(list_blue_pre0))
blue_preav1 = (sum(list_blue_pre1)/len(list_blue_pre1))   
blue_preav2 = (sum(list_blue_pre2)/len(list_blue_pre2)) 


interaction_frequencyav0 = (sum(list_interaction_frequency0)/len(list_interaction_frequency0))
interaction_frequencyav1 = (sum(list_interaction_frequency1)/len(list_interaction_frequency1))
interaction_frequencyav2 = (sum(list_interaction_frequency2)/len(list_interaction_frequency2))


freezeyav0 = (sum(list_freeze0)/len(list_freeze0))
freezeyav1 = (sum(list_freeze1)/len(list_freeze1))
freezeyav2 = (sum(list_freeze2)/len(list_freeze2))


latencyav0 = (sum(list_latency0)/len(list_latency0))
latencyav1 = (sum(list_latency1)/len(list_latency1))
latencyav2 = (sum(list_latency2)/len(list_latency2))


interactionav0 = (sum(list_interaction0_zone)/len(list_interaction0_zone))
interactionav1 = (sum(list_interaction1_zone)/len(list_interaction1_zone))
interactionav2 = (sum(list_interaction2_zone)/len(list_interaction2_zone))



df = pd.DataFrame({
    'group': [str(listgroup[0]),str(listgroup[1]),str(listgroup[2])],
    'Blue preference':[blue_preav0,blue_preav1,blue_preav2],
    'Freezing':[freezeyav0,freezeyav1,freezeyav2],
    'Interaction zone':[interactionav0,interactionav1,interactionav2],
    'Interaction count':[interaction_frequencyav0,interaction_frequencyav1,interaction_frequencyav2],
    'Latency':[latencyav0,latencyav1,latencyav2]
    })


# number of variable
categories=list(df)[1:]
N = len(categories)
 
# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]
 
# Initialise the spider plot
ax = plt.subplot(111, polar=True)
 
# If you want the first axis to be on top:
ax.set_theta_offset(pi / 2)
ax.set_theta_direction(-1)
 
# Draw one axe per variable + add labels labels yet
plt.xticks(angles[:-1], categories)
 
# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([20,40,60,80,100], ["20",'40','60','80','100'], color="grey", size=7)
plt.ylim(0,100)
 

# Ind2
values=df.loc[2].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label=str(listgroup[2]))
ax.fill(angles, values, 'r', alpha=0.00)




# Ind4
values=df.loc[1].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label=str(listgroup[1]))
ax.fill(angles, values, 'r', alpha=0.00)

# Ind1
values=df.loc[0].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label=str(listgroup[0]))
ax.fill(angles, values, 'b', alpha=0.0)

 
# Add legend
plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1.12));

plt.savefig('Desktop/Comfort_zone_paradigm/data/spider_pomc.png', dpi=400);

In [ ]:
filter_option = 'Treatment'

spiderbasal = spider
spiderbasal = spiderbasal[spiderbasal.Condition == 'Basal']
spiderbasal = spiderbasal[spiderbasal.Strain != 'Pomc:bPAC-']
spiderbasal = spiderbasal[spiderbasal.Strain != 'Pomc:bPAC+']
spiderbasal = spiderbasal[spiderbasal.Strain != 'Star:bPAC-']
spiderbasal = spiderbasal[spiderbasal.Strain != 'Star:bPAC+']
spiderbasal = spiderbasal[spiderbasal.date != '11.04.2020']
spiderbasal = spiderbasal[spiderbasal.Treatment == 'None']
spiderbasal = spiderbasal[spiderbasal.interaction_zone > 16]
spiderbasal = spiderbasal.reset_index(drop=True)
print(len(spiderbasal))

grouped = (spiderbasal.groupby('Strain').groups)
#grouped = (spiderbasal.groupby('Treatment').groups)


listgroup=[]

for name in grouped:
    group=name
    listgroup.append(group)

list_blue_pre0,list_blue_pre1,list_blue_pre2,list_blue_pre3,list_social0,list_social1,list_social2,list_social3,list_shoal0,list_shoal1,list_shoal2,list_shoal3=[],[],[],[],[],[],[],[],[],[],[],[]
list_blue0,list_blue1,list_blue2,list_blue3,list_depth_range0,list_depth_range1,list_depth_range2,list_depth_range3,list_depth_mean0,list_depth_mean1,list_depth_mean2,list_depth_mean3=[],[],[],[],[],[],[],[],[],[],[],[]
list_interaction_frequency0,list_interaction_frequency1,list_interaction_frequency2,list_interaction_frequency3,list_int_duration0,list_int_duration1,list_int_duration2,list_int_duration3=[],[],[],[],[],[],[],[]
list_int_speed0,list_int_speed1,list_int_speed2,list_int_speed3=[],[],[],[]
list_interaction0,list_interaction1,list_interaction2,list_interaction3 = [],[],[],[]
list_latency0,list_latency1,list_latency2,list_latency3,list_latency4 =[],[],[],[],[]
list_freeze0,list_freeze1,list_freeze2,list_freeze3,list_freeze4 =[],[],[],[],[]
list_interaction_frequency0,list_interaction_frequency1,list_interaction_frequency2,list_interaction_frequency3,list_interaction_frequency4 =[],[],[],[],[]
list_interaction0_zone,list_interaction1_zone,list_interaction2_zone,list_interaction3_zone,list_interaction4_zone = [],[],[],[],[]
list_blue_pre4, list_interaction_frequency4=[],[]

for i in range(0,len(spiderbasal.blue)):
    if spiderbasal.Strain[i]==listgroup[0]:
        list_blue_pre0.append(spiderbasal.blue_pre[i]) 
        list_freeze0.append(100-spiderbasal.freeze[i])
        list_interaction0_zone.append(spiderbasal.interaction_zone[i])
        list_latency0.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency0.append(0.75*(spiderbasal.int_number[i]))
    if spiderbasal.Strain[i]==listgroup[1]: 
        list_blue_pre1.append(spiderbasal.blue_pre[i]) 
        list_freeze1.append(100-spiderbasal.freeze[i])
        list_interaction1_zone.append(spiderbasal.interaction_zone[i])
        list_latency1.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency1.append(0.75*(spiderbasal.int_number[i]))



blue_preav0 = (sum(list_blue_pre0)/len(list_blue_pre0))
blue_preav1 = (sum(list_blue_pre1)/len(list_blue_pre1))   



interaction_frequencyav0 = (sum(list_interaction_frequency0)/len(list_interaction_frequency0))
interaction_frequencyav1 = (sum(list_interaction_frequency1)/len(list_interaction_frequency1))



freezeyav0 = (sum(list_freeze0)/len(list_freeze0))
freezeyav1 = (sum(list_freeze1)/len(list_freeze1))



latencyav0 = (sum(list_latency0)/len(list_latency0))
latencyav1 = (sum(list_latency1)/len(list_latency1))



interactionav0 = (sum(list_interaction0_zone)/len(list_interaction0_zone))
interactionav1 = (sum(list_interaction1_zone)/len(list_interaction1_zone))




df = pd.DataFrame({
    'group': [str(listgroup[0]),str(listgroup[1])],
    'Blue preference':[blue_preav0,blue_preav1],
    'Freezing':[freezeyav0,freezeyav1],
    'Interaction zone':[interactionav0,interactionav1],
    'Interaction count':[interaction_frequencyav0,interaction_frequencyav1],
    'Latency':[latencyav0,latencyav1]
    })


# number of variable
categories=list(df)[1:]
N = len(categories)
 
# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]
 
# Initialise the spider plot
ax = plt.subplot(111, polar=True)
 
# If you want the first axis to be on top:
ax.set_theta_offset(pi / 2)
ax.set_theta_direction(-1)
 
# Draw one axe per variable + add labels labels yet
plt.xticks(angles[:-1], categories)
 
# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([20,40,60,80,100], ["20",'40','60','80','100'], color="grey", size=7)
plt.ylim(0,100)
 

# Ind2
values=df.loc[0].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label=str('Female'))
ax.fill(angles, values, 'r', alpha=0.00)

# Ind4

values=df.loc[1].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', label=str('Male'))
ax.fill(angles, values, 'r', alpha=0.00)



 
# Add legend
plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1.12));

plt.savefig('Desktop/Comfort_zone_paradigm/data/spider_MalevsFemale.png', dpi=400);

In [ ]:
filter_option = 'Treatment'

spiderbasal = pd.read_csv('Desktop/comfort_zone_paradigm/data/Spider.csv')

#spiderbasal = spider

spiderbasal = spiderbasal[spiderbasal.date == '10.05.2020']
spiderbasal = spiderbasal[spiderbasal.Condition == 'Basal']

spiderbasal = spiderbasal.reset_index(drop=True)
print(len(spiderbasal))

grouped = (spiderbasal.groupby('Strain').groups)
#grouped = (spiderbasal.groupby('Treatment').groups)


listgroup=[]

for name in grouped:
    group=name
    listgroup.append(group)

list_blue_pre0,list_blue_pre1,list_blue_pre2,list_blue_pre3,list_social0,list_social1,list_social2,list_social3,list_shoal0,list_shoal1,list_shoal2,list_shoal3=[],[],[],[],[],[],[],[],[],[],[],[]
list_blue0,list_blue1,list_blue2,list_blue3,list_depth_range0,list_depth_range1,list_depth_range2,list_depth_range3,list_depth_mean0,list_depth_mean1,list_depth_mean2,list_depth_mean3=[],[],[],[],[],[],[],[],[],[],[],[]
list_interaction_frequency0,list_interaction_frequency1,list_interaction_frequency2,list_interaction_frequency3,list_int_duration0,list_int_duration1,list_int_duration2,list_int_duration3=[],[],[],[],[],[],[],[]
list_int_speed0,list_int_speed1,list_int_speed2,list_int_speed3=[],[],[],[]
list_interaction0,list_interaction1,list_interaction2,list_interaction3 = [],[],[],[]
list_latency0,list_latency1,list_latency2,list_latency3,list_latency4 =[],[],[],[],[]
list_freeze0,list_freeze1,list_freeze2,list_freeze3,list_freeze4 =[],[],[],[],[]
list_interaction_frequency0,list_interaction_frequency1,list_interaction_frequency2,list_interaction_frequency3,list_interaction_frequency4 =[],[],[],[],[]
list_interaction0_zone,list_interaction1_zone,list_interaction2_zone,list_interaction3_zone,list_interaction4_zone = [],[],[],[],[]
list_blue_pre4, list_interaction_frequency4=[],[]

for i in range(0,len(spiderbasal.blue)):
    if spiderbasal.Strain[i]==listgroup[0]:
        list_blue_pre0.append(spiderbasal.blue_pre[i]) 
        list_freeze0.append(100-spiderbasal.freeze[i])
        list_interaction0_zone.append(spiderbasal.latestage_interaction[i])
        list_latency0.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency0.append(0.75*(spiderbasal.int_number[i]))
    if spiderbasal.Strain[i]==listgroup[1]: 
        list_blue_pre1.append(spiderbasal.blue_pre[i]) 
        list_freeze1.append(100-spiderbasal.freeze[i])
        list_interaction1_zone.append(spiderbasal.latestage_interaction[i])
        list_latency1.append(101-((spiderbasal.Latency[i]-60)*0.84))
        list_interaction_frequency1.append(0.75*(spiderbasal.int_number[i]))



blue_preav0 = (sum(list_blue_pre0)/len(list_blue_pre0))
blue_preav1 = (sum(list_blue_pre1)/len(list_blue_pre1))   



interaction_frequencyav0 = (sum(list_interaction_frequency0)/len(list_interaction_frequency0))
interaction_frequencyav1 = (sum(list_interaction_frequency1)/len(list_interaction_frequency1))



freezeyav0 = (sum(list_freeze0)/len(list_freeze0))
freezeyav1 = (sum(list_freeze1)/len(list_freeze1))



latencyav0 = (sum(list_latency0)/len(list_latency0))
latencyav1 = (sum(list_latency1)/len(list_latency1))



interactionav0 = (sum(list_interaction0_zone)/len(list_interaction0_zone))
interactionav1 = (sum(list_interaction1_zone)/len(list_interaction1_zone))




df = pd.DataFrame({
    'group': [str(listgroup[0]),str(listgroup[1])],
    'Blue preference':[blue_preav0,blue_preav1],
    'Freezing':[freezeyav0,freezeyav1],
    'Interaction zone':[interactionav0,interactionav1],
    'Interaction count':[interaction_frequencyav0,interaction_frequencyav1],
    'Latency':[latencyav0,latencyav1]
    })


# number of variable
categories=list(df)[1:]
N = len(categories)
 
# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]
 
# Initialise the spider plot
ax = plt.subplot(111, polar=True)
 
# If you want the first axis to be on top:
ax.set_theta_offset(pi / 2)
ax.set_theta_direction(-1)
 
# Draw one axe per variable + add labels labels yet
plt.xticks(angles[:-1], categories)
 
# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([20,40,60,80,100], ["20",'40','60','80','100'], color="grey", size=7)
plt.ylim(0,100)
 

# Ind4

values=df.loc[1].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid',color = 'forestgreen', label=str('TU WT'))
ax.fill(angles, values, 'r', alpha=0.00)
    
    
# Ind2
values=df.loc[0].drop('group').values.flatten().tolist()
values += values[:1]
ax.plot(angles, values, linewidth=1, linestyle='solid', color = 'firebrick', label=str('Pomc:bPAC+'))
ax.fill(angles, values, 'r', alpha=0.00)





 
# Add legend
plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1.12));

plt.savefig('Desktop/Comfort_zone_paradigm/data/spider_MalevsFemale.png', dpi=600);

In [ ]:


abtprint(spiderbasal)